
#### Users only need to make changes to the drive variable in the functions "writetofolder" and "writetag". Remaining definitions require no changes

In [1]:
#Import all libraries needed
import pandas as pd
import os
import pydicom as dicom
import cv2
import matplotlib.pyplot as plt
import numpy as np
np.bool = np.bool_ 
np.float = np.float_
import SimpleITK as sitk
import sys
import skimage
from skimage import morphology
from skimage.segmentation import active_contour
from skimage import data, io, img_as_ubyte,filters
from skimage.measure import label
import matplotlib as mpl
import imutils 
from typing import Any
from PIL import ImageEnhance 
from PIL import Image
import math 
%matplotlib inline
mpl.rc('image', interpolation='none')
plt.rcParams['figure.figsize'] = (7.0, 7.0)
from astropy.table import QTable 
from tabulate import tabulate
from statistics import mean 
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import *
import pydicom

In [3]:

def getimages(image):
    """
    This function takes dcm images and converts to array that is then normalized  
    """
    Ivol=sitk.GetArrayFromImage(image[:image.GetSize()[0]//2,:,:])#get all slices #left, right, top, top-bottom 
    Ivol8=np.zeros([Ivol.shape[0], Ivol.shape[1], Ivol.shape[2]], dtype='uint8') 
    for i in range(Ivol.shape[0]):
        Ivol8[i]=cv2.normalize(src=Ivol[i], dst=None, alpha=0.0, beta=255.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    j=Ivol8[0].shape
    plotx=5
    return Ivol8,j,plotx
            
def stackimages(image, x=0, y=0): 
    """
    This function displays all 15 slices at once (5 by 3)
    """
    plotx = 5
    fig, axs = plt.subplots ((j//plotx), plotx, figsize=(20,10)) 
    for i in range(j):
        axs[y, x].imshow(image[i], cmap='bone')
        axs[y, x].set_title(f"slice {i+1}", fontsize=12) 
        axs[y,x].axis("off")
        if x <(plotx-1):
            x+=1
        else:
            x=0
            y+=1


In [4]:
def stackimages_w_filename(image, x=0, y=0):
    """
    This function displays all 15 slices at once with the corresponding filename (5 by 3)
    """
    plotx = 5
    fig, axs = plt.subplots ((j//plotx), plotx, figsize=(20,10)) 
    for i in range(j):
        axs[y, x].imshow(image[i], cmap='bone')
        axs[y, x].set_title(f"{files}: slice {i+1}", fontsize=12)
        axs[y,x].axis("off")
        if x <(plotx-1):
            x+=1
        else:
            x=0
            y+=1

<span style="font-size: 25px;"> Inhomogeneity Correction

#### Definitions for image correction

In [5]:
# get_numpy_DICOM_volume
def get_numpy_DICOM_volume(image_path: str) -> np.array:
    file_names = os.listdir(image_path)

    # Load DICOM images and extract z-coordinates
    slices = []
    for file_name in file_names:
        ds = pydicom.dcmread(os.path.join(image_path, file_name))
        slices.append((ds.ImagePositionPatient[2], ds.pixel_array))

    # Sort slices by z-coordinate
    slices.sort()

    # Stack 2D pixel arrays into 3D numpy array
    volume = np.stack([slice[1] for slice in slices], axis=-1)
    return volume

# show_diff
def show_diff(orig_read_data: Any, alt_read_data: Any, show_image: bool = True, show_hist: bool = False) -> dict[str, int]:
    """
    This function shows the differences between 2 images in terms of removed pixels (painted black), added pixels (black pixels painted some grey scale)
    and shifted pixels (greyscale pixels whos value changes)    
    """
    if type(orig_read_data) is str:
        ds_orig = pydicom.dcmread(orig_read_data).pixel_array
    else:
        ds_orig = orig_read_data
    if type(alt_read_data) is str:
        ds_alt = pydicom.dcmread(alt_read_data).pixel_array
    else:
        ds_alt = alt_read_data
    
    orig_img = Image.fromarray(ds_orig)
    alt_img = Image.fromarray(ds_alt)
    alt_img = alt_img.convert("L")
    orig_img = orig_img.convert("L")

    orig_numpydata = np.asarray(orig_img)
    alt_numpydata = np.asarray(alt_img)
    diff_array = np.zeros(
        (orig_numpydata.shape[0], orig_numpydata.shape[1], 3))
    color_counts = {"Red": 0, "Green": 0, "Yellow": 0}

    for i in range(orig_numpydata.shape[0]):
        for j in range(orig_numpydata.shape[1]):
            if orig_numpydata[i, j] == alt_numpydata[i, j]:  
                diff_array[i, j] = [0, 0, 0]
            # non-black pixel was painted black (removed)
            elif (orig_numpydata[i, j]) != 0 and (alt_numpydata[i, j]) == 0:
                diff_array[i, j] = [255, 0, 0]
                color_counts["Red"] += 1
            # pixel color was changed (altered)
            elif (orig_numpydata[i, j]) != 0 and (alt_numpydata[i, j]) != 0:
                diff_array[i, j] = [255, 255, 0]
                color_counts["Yellow"] += 1
            # black pixel was painted a differnt color (added)
            elif (orig_numpydata[i, j]) == 0 and (alt_numpydata[i, j]) != 0:
                diff_array[i, j] = [0, 255, 0]
                color_counts["Green"] += 1
    txt = "Red: pixel value became 0 (black),\n Green: pixel 0 (black) changed to some greyscale value > 0,\n Yellow: one non-0 (non-black) greyscale value changed to another non-0 (non-black) greyscale value"
    if show_image:
        overlay = np.ma.masked_where(diff_array == 0, diff_array)
        plt.figtext(0.5, 0.01, txt, wrap=True,
                    horizontalalignment='center', fontsize=12)
        plt.axis('off')
        plt.imshow(orig_img, cmap="bone", vmin=0, vmax=2200)
        plt.imshow(overlay, vmin=1, vmax=10, alpha=0.5)
        plt.show()
        plt.clf()

    if show_hist:
        colors = list(color_counts.keys())
        values = list(color_counts.values())
        # creating the bar plot
        plt.bar(colors, values, color=colors,
                width=0.4)

        plt.xlabel(txt, wrap=True, fontsize=8)
        plt.ylabel("Count")
        plt.show()
        plt.clf()

    return color_counts


def DICOM_volume_to_numpy(folder_path: str) -> tuple[np.array, list[str]]:
    if not os.path.exists(folder_path):
        raise FileNotFoundError
    slices = []
    file_names = []
    for file in os.listdir(folder_path):
        full_path = os.path.join(folder_path, file)
        if os.path.isdir(full_path) or file.lower().endswith((".nrrd", ".ds_store")): # skip nested folders or nrrd files
            continue
        else:
            ds = pydicom.dcmread(full_path)
            slices.append((ds.ImagePositionPatient[2], ds.pixel_array))
            file_names.append(file)

    # Stack 2D pixel arrays into 3D numpy array
    volume = np.stack([slice[1] for slice in slices], axis=-1)
    return (volume, file_names)
                    
# inhomogeneity_correction

def inhomogeneity_correction(read_data: Any, filename: str, save_path: str = None, mask_array = None, show_images: bool = False) -> tuple[np.array, str]:
    
    """
    This functions corrects image inhomogeneity by means of using N4 bias field correction, this function is slow and not reccomended as 
    the brightness correction function does a much better and faster job.
    """
    if type(read_data) is str:
        ds = pydicom.dcmread(read_data)
        read_data = ds.pixel_array
        
    image = sitk.GetImageFromArray(read_data)
    image = sitk.Cast(image, sitk.sitkFloat32)
    if mask_array is None:
        maskImage = sitk.OtsuThreshold(image, 0, 1, 200)
    else:
        mask_image = sitk.GetImageFromArray(mask_image)

    corrector = sitk.N4BiasFieldCorrectionImageFilter()
    corrected_image = corrector.Execute(image, maskImage)
    corrected_image = sitk.GetArrayFromImage(corrected_image)

    log_bias_field = corrector.GetLogBiasFieldAsImage(image)

    corrected_image_full_resolution = image / sitk.Exp(log_bias_field)
    corrected_image = sitk.GetArrayFromImage(corrected_image_full_resolution)
    corrected_image = cv2.normalize(src=corrected_image, dst=None, alpha=0.0, beta=255.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_16U)
    
    if show_images:
        plt.imshow(corrected_image, cmap='bone')
        plt.show()

    return (corrected_image, f"i_c-{filename}")


def sharpen1(read_data: Any, filename: str = None, save_path: str = None, mode: str = "e_e", show_images: bool = False) -> tuple[np.array, str]:
    """
    Function that applies 3 different kernels that sharpen an image
    show_images defaults to False, setting to True will display the result.

    Supports modes: "s", "e_s", "e_e"
    These modes corrsepond to sharpening, exessive sharpening, edge enhancement respectivley. "e_e" is applied by default
    """
    # Retrieved from https://subscription.packtpub.com/book/application-development/9781785283932/2/ch02lvl1sec22/sharpening

    if type(read_data) is str:
        ds = pydicom.dcmread(read_data)
        read_data = ds.pixel_array
    if mode == "s":
        # generating the kernels
        kernel_sharpen_1 = np.array([[-1, -1, -1],
                                    [-1, 9, -1],
                                    [-1, -1, -1]])
        output = cv2.filter2D(read_data, -1, kernel_sharpen_1)
    elif mode == "e_s":
        kernel_sharpen_2 = np.array([[1, 1, 1],
                                    [1, -7, 1],
                                    [1, 1, 1]])
        output = cv2.filter2D(read_data, -1, kernel_sharpen_2)
    elif mode == "e_e":
        kernel_sharpen_3 = np.array([[-1, -1, -1, -1, -1],
                                    [-1, 2, 2, 2, -1],
                                    [-1, 2, 100, 2, -1],
                                    [-1, 2, 2, 2, -1],
                                    [-1, -1, -1, -1, -1]]) / 2
        output = cv2.filter2D(read_data, -1, kernel_sharpen_3)
  
    if show_images:
        plt.imshow(output, cmap="bone")
        plt.show()
    return (output, f"{mode}-{filename}")

# blur

def blur(read_data: Any, mode: str = "b_b", strength: int = 5, filename: str = None, save_path: str = None, show_images: bool = False) -> tuple[np.array, str]:
    """
    This functions blurs the image with customizable strength.
    The modes supported are "b_b" and "m_b", corresponding to bilateral bluring and median bluring. 

    Bilaterl bluring keeps edges intact, typically only affecting the interior of the image, whereas median bluring affects the entire image.
    """
    if type(read_data) is str:
        ds = pydicom.dcmread(read_data)
        read_data = ds.pixel_array
    
    read_data = cv2.normalize(
        read_data, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    if mode == "m_b":
        blur = cv2.medianBlur(read_data, strength)
    elif mode == "b_b":
        blur = cv2.bilateralFilter(read_data, d=strength * 5, sigmaColor=10,sigmaSpace=75)
  
    if show_images:
        plt.imshow(blur, cmap="bone")
        plt.show()
    return (blur, f"{mode}-{filename}")

# thresholding


def thresholding(read_data: Any, strength: int = 55, filename: str = None, save_path: str = None, show_images: bool = False) -> tuple[np.array, str]:
    """
    This function applies uniform thresholding based on a strength value, it is set up to support other modes of thresholding, but they 
    are not currently in use.
    """

    if type(read_data) is str:
        ds = pydicom.dcmread(read_data)
        read_data = ds.pixel_array

    read_data = cv2.normalize(
        read_data, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    _, th1 = cv2.threshold(read_data, strength, 255, cv2.THRESH_BINARY)
   
    if show_images:
        plt.imshow(th1, cmap="bone")
        plt.show()
    return (th1, f"t-{filename}")

# fill_contours


def fill_contours(read_data: str, filename: str = None, dilation_iterations: int = 2, crop: bool = False, save_path: str = None, show_images: bool = False) -> tuple[np.array, str]:
    """
    This function fills in the interior of the thighs and also supports cropping images who have had their border increased by 10 pixels.
    """
    if type(read_data) is str:
        ds = pydicom.dcmread(read_data)
        read_data = ds.pixel_array

    img_uint8 = cv2.normalize(
        read_data, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    contours, _ = cv2.findContours(
        img_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)


    largest_contours = sorted([cv2.contourArea(obj) for obj in contours], reverse=True)[:2]
    # only grab the two largest contours corresponding to the thigh slices
    thighs = [obj for obj in contours if cv2.contourArea(obj) in largest_contours]

    if len(thighs) > 1 and max(largest_contours) - min(largest_contours) > 0.9 * max(largest_contours):
        thighs = [contour for contour in thighs if cv2.contourArea(contour) == max(largest_contours)]

    background = np.zeros(read_data.shape)
    mask = cv2.fillPoly(background, thighs, color=(255, 255, 255))
    # Define kernel for dilation
    kernel = np.ones((3, 3), np.uint8)

    # Dilate mask by 1 pixel
    dilated_mask = cv2.dilate(mask, kernel, iterations=dilation_iterations)

    dilated_mask_uint8 = cv2.normalize(
        dilated_mask, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    contours, _ = cv2.findContours(dilated_mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    h, w = dilated_mask_uint8.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        surrounding_pixels = dilated_mask_uint8[y - 1:y + h + 1, x - 1:x + w + 1]
        if np.all(surrounding_pixels == 255):
        # Apply flood fill to fill the black hole
            cv2.floodFill(dilated_mask_uint8, mask, (x, y), 255)
    
    if crop:
        dilated_mask_uint8 = dilated_mask_uint8[10:266, 10:522]

    if show_images:
        plt.imshow(dilated_mask_uint8, cmap="bone")
        plt.show()
    return (dilated_mask_uint8, f"f_c-{filename}")
        
    
# denoise


def denoise(read_data: Any, filename: str = None, save_path: str = None, show_images: bool = False) -> tuple[np.array, str]:
    """
    This function denoises an image by using a variety of morphological transformations as well as increaseing the border size by 10 pixels,
    this is needed to accomodate for some of the transformations. Filling the contours at some point after using this function will correct for the added pixels.
    """
    if type(read_data) is str:
        ds = pydicom.dcmread(read_data)
        read_data = ds.pixel_array

    read_data = cv2.copyMakeBorder(src=read_data, top=10, bottom=10, left=10, right=10, borderType=cv2.BORDER_CONSTANT) 

    read_data_bin = read_data > 0
    output = skimage.morphology.closing(read_data_bin)
    output = skimage.morphology.binary_dilation(output, np.ones((5,5)))
    output = skimage.morphology.remove_small_objects(
        output, min_size=1000, connectivity=1)
    
    output = (output * 1).astype('uint8')

    output = fill_contours(read_data=output, dilation_iterations=5)
    output = read_data > 0
    
    output = skimage.morphology.binary_opening(output)
    kernel_erosion_small = np.array([ [0, 1, 1, 0],
                                      [0, 1, 1, 0],
                                      [0, 1, 1, 0],
                                      [0, 1, 1, 0]])
    output = skimage.morphology.binary_erosion(output, kernel_erosion_small)
    output = (output * 1).astype('uint8')

    output = fill_contours(read_data=output, dilation_iterations=3)
    
             
    output = output[0] > 0
    kernel_erosion_large = np.array([   [-1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, -1],
                                        [-1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, -1],
                                        [-1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, -1],
                                        [-1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, -1],
                                        [-1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, -1]])
    output = skimage.morphology.binary_erosion(output, kernel_erosion_large)
    kernel_erosion_small = np.array([ [0, 1, 0],
                                      [0, 1, 0]])
    output = skimage.morphology.binary_erosion(output, kernel_erosion_small)
    output = (output * 1).astype('uint8')
    kernel = np.ones((3, 3), np.uint8)
    output = cv2.erode(output, kernel, iterations=4)
    
    if show_images:
        plt.imshow(output, cmap="bone")
        plt.show()
    return (output, f"d-{filename}")    

# histogram_normalization

def brightness_correction(read_data: Any, mode: str = "CLAHE", strength: int = 2, ds: pydicom.Dataset = None, filename: str = None, save_path: str = None, show_images: bool = False) -> tuple[np.array, str, Any]:
    """
    This function corrects the brightness in an image by either using CLAHE or uniform brightening in order to address dark spots and general inhomogeneity
    """
    if type(read_data) is str:
        ds = pydicom.dcmread(read_data)
        read_data = ds.pixel_array
    img_uint8 = cv2.normalize(read_data, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    non_black_white_pixels = None
    gray = np.uint8(img_uint8 / np.max(img_uint8) * 255)
    if mode == "CLAHE":
        clahe = cv2.createCLAHE(clipLimit= strength, tileGridSize=(11,11))
        result = clahe.apply(gray)
    elif mode == "BI":
        non_black_white_pixels = np.logical_and(gray > 10, gray < 30)
        # Increase brightness for non-black pixels
        brightened_image = np.clip(np.where(non_black_white_pixels, img_uint8 + strength, img_uint8), 0, 255)
        # Clip the values to ensure they remain within the valid range [0, 255]
        result = brightened_image
    result = cv2.normalize(result, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_16U)
    if show_images:
        plt.imshow(result, cmap="bone")
        plt.show()
    return (result, f"{mode}-{filename}", non_black_white_pixels)


def get_mask(read_data: Any) -> np.array:
    """
    This function retruns all the pixels in an image that are white
    """
    if type(read_data) is str:
        ds = pydicom.dcmread(read_data)
        img = ds.pixel_array
    else:
        img = read_data
    white_pixels = np.argwhere(img == 255)
    return white_pixels


def get_cleaned_image(read_data: Any, mask: Any, ds: pydicom.Dataset = None, filename: str = None, save_path: str = None, show_images: bool = False) -> np.array:
    """
    This function gets a cleaned image based on a base/corrected image as well as a mask containing all the pixels which we wish to retain in the new image.
    """
    
    if type(mask) is str:
        ds = pydicom.dcmread(mask)
        mask_array = ds.pixel_array
    else:
        mask_array = mask

    binary_array = np.where(mask_array == 255, 1, mask_array)
    if type(read_data) is str:
        ds = pydicom.dcmread(read_data)
        read_array = ds.pixel_array
    else:
        read_array = read_data
    combined = read_array*binary_array 
    combined = cv2.normalize(combined, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_16U)
    
    if show_images:
        plt.imshow(combined, cmap="bone")
        plt.show()
    return combined



<span style="font-size: 25px;"> Leg Separation

In [7]:
def BackupLegSeparation(image, side): 
    '''
    This function runs backup leg model and resets i so all slices use backup version
    '''
    pt_image = image
    pixel_values = np.sum(pt_image, axis=0) #find which x values have brightest pixels
    from skimage.filters import threshold_otsu, threshold_multiotsu
    x_threshold = threshold_otsu(pixel_values)
    x_coordinates = np.arange(len(pixel_values)) #assign x values

    # Save the data as a NumPy array
    data_array = np.array([x_coordinates, pixel_values])
    np.save('data_array.npy', data_array)
    loaded_array = np.load('data_array.npy')

    x_coordinates = np.arange(len(loaded_array[1]))

    #counting intersections to determine range of interest
    def count_intersections(x_data,y_data, horizontal_line_y):
        num_intersections = 0

        for i in range(len(y_data) - 1):
            y1, y2 = y_data[i], y_data[i + 1]
            x1, x2 = x_data[i], x_data[i + 1]

            # Check if the horizontal line intersects the line segment
            if min(y1, y2) <= horizontal_line_y <= max(y1, y2) and y1 != y2:
                x_intersection = x1 + (horizontal_line_y - y1) * (x2 - x1) / (y2 - y1)
                if x1 <= x_intersection <= x2:
                    num_intersections += 1

        return num_intersections

    intersection_count = np.array([])
    for i in range (max(loaded_array[1])-1):
        horizontal_line_y = i
        # Count intersections
        intersections = count_intersections(x_coordinates, loaded_array[1], horizontal_line_y)
        intersection_count = np.append(intersection_count, intersections)

    #finding the target Y value
    value_to_find = 4
    indices = np.where(intersection_count == value_to_find)[0]
    target_y = np.median(indices)


    # Function to find X values where a vertical line intersects the graph at a specific Y value
    def find_vertical_line_intersections(x_data, y_data, target_y):
        intersections_x = []

        for i in range(len(y_data) - 1):
            y1, y2 = y_data[i], y_data[i + 1]
            x1, x2 = x_data[i], x_data[i + 1]

            # Check if the vertical line intersects the line segment
            if min(y1, y2) <= target_y <= max(y1, y2) and y1 != y2:
                x_intersection = x1 + (target_y - y1) * (x2 - x1) / (y2 - y1)
                if x1 <= x_intersection <= x2:
                    intersections_x.append(x_intersection)

        return intersections_x

    # Call the function to find X values
    intersections_x = find_vertical_line_intersections(x_coordinates, loaded_array[1], target_y)

    #cut image at average between 2 middle intersections x values
    mid = round((intersections_x[1] + intersections_x[2])/2) #x-value between legs

    desired_shape = (256, 256)  # define desired shape (rows, columns)
    #Slicing image at midpoint
    #left side
    if side == 'L':
        centered_image = pt_image[:, :mid]
        np.expand_dims(centered_image,axis=2)
        if desired_shape[1] - centered_image.shape[1] > 0: #add columns
            num_columns_to_add = desired_shape[1] - centered_image.shape[1]
            centered_image_f = np.pad(centered_image, ((0, 0), (0, num_columns_to_add)), mode='constant')
        elif desired_shape[1] - centered_image.shape[1] < 0: #remove columns
            num_columns_to_remove = centered_image.shape[1] - desired_shape[1]
            centered_image_f = centered_image[:, num_columns_to_remove:]
        else:
            centered_image_f = centered_image
    #right side
    if side == 'R':
        centered_image = pt_image[:, mid:]
        np.expand_dims(centered_image,axis=2)
        if desired_shape[1] - centered_image.shape[1] > 0: #add columns
            num_columns_to_add = desired_shape[1] - centered_image.shape[1]
            centered_image_f = np.pad(centered_image, ((0, 0), (num_columns_to_add,0)), mode='constant')
        elif desired_shape[1] - centered_image.shape[1] < 0: #remove columns
            num_columns_to_remove = centered_image.shape[1] - desired_shape[1]
            centered_image_f = centered_image[:, :desired_shape[1]]
        else:
            centered_image_f = centered_image
    def first_last_nonzero_indices(arr):
        # Find the first nonzero index
        first_nonzero = next((index for index, value in enumerate(arr) if value != 0), None)
        # Find the last nonzero index
        last_nonzero = next((index for index, value in reversed(list(enumerate(arr))) if value != 0), None)
        #print(f'{first_nonzero} and {last_nonzero}')
        return first_nonzero, last_nonzero

    pixel_values = np.sum(centered_image_f, axis=0) #use new image
    first, last = first_last_nonzero_indices(pixel_values)
    current_mid = round((first - last)/2)
    if side == 'L':
        shift = 128 - current_mid
    if side == 'R':
        shift = -1* (128 - current_mid) 
    shifted_array = np.roll(centered_image_f, shift, axis=1)
    return centered_image_f



In [16]:
def LegSepNew (image, side, rmv_count, num_columns_to_remove):
    '''
    This function separates the thighs by side so each resulting image only contains one thigh
    '''
    read_data = image
    num_columns_to_remove = num_columns_to_remove
    rmv_count = rmv_count
    read_data = cv2.normalize(read_data, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    kernel = np.array([[0, 1, 0],
                [1, 1, 1],
                [0, 1, 0]], dtype = np.uint8)
    erode = cv2.erode(read_data, kernel, iterations=1)
    blur = cv2.medianBlur(erode, 3)
    blur = cv2.bilateralFilter(blur,9,10,50)                                               #locating 2 large bright regions (legs)
    ret, thresh = cv2.threshold(blur, 30, 255,cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    largest_contours = sorted([cv2.contourArea(obj) for obj in contours], reverse=True)[:2]
    thighs = [obj for obj in contours if cv2.contourArea(obj) in largest_contours]
    backtorgb = cv2.cvtColor(thresh,cv2.COLOR_GRAY2RGB)
    cv2.drawContours(backtorgb, [thighs[0]], -1, (255,0,0), 1)
    cv2.drawContours(backtorgb, [thighs[1]], -1, (0,0,255), 1)
    min_centroid = ()
    max_centroid = ()
    if side == 'L':
            for contour in thighs:
                # Calculate the centroid of the contour
                M = cv2.moments(contour)
                cX = int(M['m10'] / M['m00'])
                cY = int(M['m01'] / M['m00'])
                if len(min_centroid) == 0 or cX < min_centroid[0]:     #chooses centroid with smaller x value (i.e. left side)
                    min_centroid = (cX, cY, contour)
            min_contour = min_centroid[2]
            end_x = min_contour[:, :, 0].max() # Find the ending x-coordinate (rightmost point) of the contour
            if rmv_count > 0:
                centered_image = read_data[:,num_columns_to_remove:end_x]
                start_x = min_contour[:, :, 0].min()
                
                if start_x < num_columns_to_remove:
                    print("using backup")
                    centered_image = BackupLegSeparation(read_data, "L")
            else:
                centered_image = read_data[:, :end_x] #cut off at centroid
            
            #reform new shape
            desired_shape = (256, 256)  # define desired shape (rows, columns)
            np.expand_dims(centered_image,axis=2)
            if desired_shape[1] - centered_image.shape[1] > 0: #add columns
                num_columns_to_add = desired_shape[1] - centered_image.shape[1]
                centered_image_f = np.pad(centered_image, ((0, 0), (0, num_columns_to_add)), mode='constant')
            elif desired_shape[1] - centered_image.shape[1] < 0: #remove columns
                num_columns_to_remove = centered_image.shape[1] - desired_shape[1]
                centered_image_f = centered_image[:,:desired_shape[1]]
                print("Too Big")
            else:
                centered_image_f = centered_image              
                         
                
    if side == 'R':
        max_cX = None
        for contour in thighs:
            # Calculate the centroid of the contour
            M = cv2.moments(contour)
            cX = int(M['m10'] / M['m00'])
            cY = int(M['m01'] / M['m00'])
            if max_cX is None or cX > max_cX:
                max_cX = cX  # Update max_cX with the current cX
                max_centroid = (cX, cY, contour) #pick right side

        max_contour = max_centroid[2]
        start_x = max_contour[:, :, 0].min()# Find the ending x-coordinate (leftmost point) of the contour
        desired_shape = (256, 256)  # define desired shape (rows, columns)
        if rmv_count > 0:
            cutoff = 512 - num_columns_to_remove
            centered_image = read_data[:,start_x:cutoff]
            start_x = max_contour[:, :, 0].max()
        else:
            centered_image = read_data[:, start_x:]
        
        if desired_shape[1] - centered_image.shape[1] > 0: #add columns
            num_columns_to_add = desired_shape[1] - centered_image.shape[1]
            centered_image_f = np.pad(centered_image, ((0, 0), (num_columns_to_add,0)), mode='constant')
        elif desired_shape[1] - centered_image.shape[1] < 0: #remove columns
            num_columns_to_remove = centered_image.shape[1] - desired_shape[1]
            centered_image_f = centered_image[:, num_columns_to_remove:]
        else:
            centered_image_f = centered_image
            
    return centered_image_f
  
def GetShift(image,side):
    '''
    This function finds how much to shift all slices in patient so the final images contains entire thigh regardless of size and each slice is still in the same position relative to the slices beside it (z-connectivity)
    '''
    read_data = image
    read_data = cv2.normalize(read_data, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    kernel = np.array([[0, 1, 0],
                [1, 1, 1],
                [0, 1, 0]], dtype = np.uint8)
    erode = cv2.erode(read_data, kernel, iterations=1)
    blur = cv2.medianBlur(erode, 3)
    blur = cv2.bilateralFilter(blur,9,10,50)                                               #locating 2 large bright regions (legs)
    ret, thresh = cv2.threshold(blur, 30, 255,cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    largest_contours = sorted([cv2.contourArea(obj) for obj in contours], reverse=True)[:2]
    thighs = [obj for obj in contours if cv2.contourArea(obj) in largest_contours]
    backtorgb = cv2.cvtColor(thresh,cv2.COLOR_GRAY2RGB)
    cv2.drawContours(backtorgb, [thighs[0]], -1, (255,0,0), 1)
    cv2.drawContours(backtorgb, [thighs[1]], -1, (0,0,255), 1)
    min_centroid = ()
    max_centroid = ()
    if side == 'L':
            for contour in thighs:
                # Calculate the centroid of the contour
                M = cv2.moments(contour)
                cX = int(M['m10'] / M['m00'])
                cY = int(M['m01'] / M['m00'])
                if len(min_centroid) == 0 or cX < min_centroid[0]:     #chooses centroid with smaller x value (i.e. left side)
                    min_centroid = (cX, cY, contour)
            min_contour = min_centroid[2]
            end_x = min_contour[:, :, 0].max() # Find the ending x-coordinate ( rightmost point) of the contour
            rmv_count = 0 #care to know if we removed columns
            desired_shape = (256, 256)  # define desired shape (rows, columns)
            centered_image = read_data[:, :end_x] #cut off at centroid
            np.expand_dims(centered_image,axis=2)
            if desired_shape[1] - centered_image.shape[1] < 0: #remove columns
                num_columns_to_remove = centered_image.shape[1] - desired_shape[1]
                centered_image_f = centered_image[:,num_columns_to_remove:] #want to remove columns at beginning
                rmv_count += 1   
            else:
                num_columns_to_remove = 0

                    
    if side == 'R':
        max_cX = None
        for contour in thighs:
            # Calculate the centroid of the contour
            M = cv2.moments(contour)
            cX = int(M['m10'] / M['m00'])
            cY = int(M['m01'] / M['m00'])
            if max_cX is None or cX > max_cX:
                max_cX = cX  # Update max_cX with the current cX
                max_centroid = (cX, cY, contour) #pick right side

        max_contour = max_centroid[2]
        end_x = max_contour[:, :, 0].min()# Find the ending x-coordinate (leftmost point) of the contour
        #print(end_x)
        rmv_count = 0 #care to know if we removed columns
        desired_shape = (256, 256)  # define desired shape (rows, columns)
        centered_image = read_data[:, :end_x] #cut off at centroid
        np.expand_dims(centered_image,axis=2)

        centered_image = read_data[:, end_x:]
        np.expand_dims(centered_image,axis=2)
        
        if desired_shape[1] - centered_image.shape[1] < 0: #remove columns
            num_columns_to_remove = centered_image.shape[1] - desired_shape[1]
            centered_image_f = centered_image[:, :desired_shape[1]] #want to remove columns at the end
            rmv_count += 1
        else:
            num_columns_to_remove = 0
     
    return rmv_count, num_columns_to_remove
    

<span style="font-size: 25px;"> Definitions for Muscle Mask

In [51]:
def enlarge_size(image,pixels): 
    '''
    This function adds a certain number of pixels to all sides of the image
    '''
    image_big =np.zeros([image.shape[0], image.shape[1]+(pixels*2), image.shape[2]+(pixels*2)], dtype='uint8')
    
    #adding pixels for shape[1] is for top_bottom - acconting for addition top bottom addition increase in size
    #adding pixels for shape[2] is for left right - acconting for addition left bottom addition increase in size
    
    pixels=pixels
    for i in range(j):
        top_bottom=np.zeros([pixels, image.shape[2]], dtype='uint8')   
        top_added=np.vstack((top_bottom,image[i])) 
        bottom_added=np.vstack((top_added,top_bottom)) 
        right_left=np.zeros([bottom_added.shape[0], pixels], dtype='uint8')  
        right_added=np.hstack((bottom_added,right_left))
        left_added=np.hstack((right_left,right_added))
        image_big[i]=left_added
    return  image_big, pixels #want pixels for reducing size later


In [53]:
def reduce_size(image_big): 
    '''
    This function removes a certain number of pixels from all sides of the image
    '''
    top_removed=np.delete(image_big, np.s_[:pixels], 1)  
    bottom_removed=np.delete(top_removed, np.s_[top_removed.shape[1]-pixels:], 1) 
    left_removed=np.delete(bottom_removed, np.s_[:pixels], 2) 
    right_removed=np.delete(left_removed, np.s_[left_removed.shape[2]-pixels:], 2)  
    image_reg=right_removed.copy()
    return image_reg

def multi_otsu_1(image):
    '''
    This function takes the Otsu threshold between the second and third class and returns the fat threshold
    '''
    motsuth=filters.threshold_multiotsu(image, classes=3)
    regions=np.digitize(image,bins=motsuth)
    output=img_as_ubyte(regions)
    return motsuth[1] 

####  Get Rough Muscle Mask

In [54]:
def floodfill(image): 
    '''
    This function flood fills the rough muscle mask
    '''
    ff_im=np.zeros([image.shape[0], image.shape[1]], dtype='uint8') 
    to_ff=image.copy()
         
    h, w = image.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)

    cv2.floodFill(to_ff, mask, (0,0), 1);
    th, ff_im = cv2.threshold(to_ff, 0, 1, cv2.THRESH_BINARY_INV)

    return ff_im

def multi_otsu_0(image):
    '''
    This function takes the Otsu threshold between the first and second class and returns the fat+muscle threshold
    '''
    motsuth=filters.threshold_multiotsu(image, classes=3)
    regions=np.digitize(image,bins=motsuth)
    output=img_as_ubyte(regions)
    return motsuth[0] 



def get_musc_fat_mask(image):
    '''
    This function applies a threshold to the image and gets the rough muscle+fat mask
    '''
    mask =np.zeros([image.shape[0], image.shape[1], image.shape[2]], dtype='uint8')
    
    k=1
    
    for i in range(j):
        th=multi_otsu_0(image[i]) 
        mask[i]=(image[i]>th) 

        mask[i] = cv2.morphologyEx(mask[i], cv2.MORPH_OPEN, np.ones((5,5),np.uint8)) #remove connections with other leg

        
        mask[i]=cv2.morphologyEx(mask[i], cv2.MORPH_CLOSE, np.ones((2,2),np.uint8))

        mask[i] = (morphology.remove_small_holes(label(mask[i]),area_threshold=9000, connectivity=1)) #remove holes to get entire mask of muscle + subcfat region

        mask[i] = cv2.morphologyEx(mask[i], cv2.MORPH_OPEN, np.ones((7,7),np.uint8)) #get rid of line artifact

        mask[i] = (morphology.remove_small_objects(label(mask[i]),min_size=3000, connectivity=1))#get rid of noise 

        ret, mask[i] = cv2.threshold(mask[i],0,1,cv2.THRESH_BINARY) 
        
    return mask


def get_subcfat_ring(image):
    '''
    This function finds the subcutaneous fat ring which is used to close the gaps for floodfilling when isolating muscle
    '''
    subcfat_ring=np.zeros([image.shape[0], image.shape[1], image.shape[2]], dtype='uint8')
    for i in range(j):
        eroded=cv2.erode(image[i],np.ones((5,5),np.uint8),iterations = 1) 
        subcfat_ring[i]=image[i]-eroded 
    return subcfat_ring

#### Apply Filters

In [55]:
def CurvatureFlowImageFilter(image): 
    '''
    This function removes noise to make the fascia boundary more clear
    '''
    inputImage=sitk.GetImageFromArray(image)
    inputImage = sitk.Cast(inputImage, sitk.sitkFloat32)
    corrector = sitk.CurvatureFlowImageFilter()
    corrector.SetNumberOfIterations( 15 );
    corrector.SetTimeStep( 0.1 )
    output = corrector.Execute( inputImage)
    image_c= sitk.GetArrayFromImage(output)
    image_c=cv2.normalize(src=image_c, dst=None, alpha=0.0, beta=255.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U) 
    return image_c

def apply_CurvatureFilter(image1,image2):
    '''
    This function is used to overcome instances where the image is dark
    '''
    mask=np.zeros([image1.shape[0], image1.shape[1], image1.shape[2]], dtype='uint8')
    image=np.zeros([image1.shape[0], image1.shape[1], image1.shape[2]], dtype='uint8')
    for i in range(j):
        mask[i]=Ivol8c_subcfat[i]>multi_otsu_1(Ivol8c_subcfat[i])  
        if np.sum(mask[i]>0)<1000:
            print(f"Slice {i+1} use Ivol8c_coi")
            image[i]=image2[i]
        else:
            image[i]=image1[i]
    
    curvature_flow=np.zeros([image1.shape[0], image1.shape[1], image1.shape[2]], dtype='uint8')
    
    for i in range(j):
        curvature_flow[i]=CurvatureFlowImageFilter(image[i])
        
    return curvature_flow



In [56]:
def enhance_sharpness(image): 
    '''
    This function is used to make the fascia boundary line darker
    '''
    from PIL import ImageEnhance
    from PIL import Image

    Ivol8c_c_s=np.zeros([image.shape[0], image.shape[1], image.shape[2]], dtype='uint8')

    for i in range(j):
        f=image[i].copy()
        f=Image.fromarray(f, mode=None)
        enhancer = ImageEnhance.Sharpness(f)
        factor =  12.0 
        Ivol8c_c_s[i]=enhancer.enhance(factor)
    return Ivol8c_c_s

def get_subcfatvol(image):   
    '''
    This function gives the subcutaneous fat volume
    '''
    subcfatvol=np.zeros([image.shape[0], image.shape[1], image.shape[2]], dtype='uint8')
    
    for i in range(j):
        subcfatvol[i]=image[i]>multi_otsu_1(image[i])  

        subcfatvol[i] = (morphology.remove_small_holes(subcfatvol[i],area_threshold=90, connectivity=1))  
         
    return subcfatvol

In [57]:
def floodfillall(image):
    '''
    This function floodfills objects in a 3D image
    '''
    floodfilled=np.zeros([Ivol8c_subcfat.shape[0], Ivol8c_subcfat.shape[1], Ivol8c_subcfat.shape[2]], dtype='uint8')
    for i in range(j): 
        im_floodfill = image[i].copy()
        h, w = im_floodfill.shape[:2]
        mask = np.zeros((h+2, w+2), np.uint8)
        cv2.floodFill(im_floodfill, mask, (0,0), 1);
        th, floodfilled[i] = cv2.threshold(im_floodfill, 0, 1, cv2.THRESH_BINARY_INV)
    return floodfilled

def darkpieces(image): 
    '''
    This function is used to find the dark parts of our image including vessels
    '''
    musc_mask1=floodfillall(image) 
    musc_mask2=np.zeros([image.shape[0], image.shape[1], image.shape[2]], dtype='uint8')


    for i in range(j):
        musc_mask2[i] = cv2.morphologyEx(musc_mask1[i], cv2.MORPH_OPEN, np.ones((3,3),np.uint8)) 
        
        musc_mask2[i] = (morphology.remove_small_objects(label(musc_mask2[i]),min_size=100, connectivity=1)) 
            
        musc_mask2[i]= cv2.dilate(musc_mask2[i],np.ones((3,3),np.uint8),iterations = 1)
        musc_mask2[i] = (morphology.remove_small_objects((musc_mask2[i]).astype(bool),min_size=100, connectivity=1)) 
        musc_mask2[i]= cv2.erode(musc_mask2[i],np.ones((3,3),np.uint8),iterations = 1) 
            
        th, musc_mask2[i]= cv2.threshold(np.uint8(musc_mask2[i]), 0, 1, cv2.THRESH_BINARY)
    
    return musc_mask1, musc_mask2

#### Remove any muscle overshoots into subcfat

In [58]:
def remove_subcfat_overshoot(image1,image2,removal_round):
    '''
    This function removes any overshooting that may occur when creating subcutaneous fat mask
    '''
    a=image1*image2
    
    
    overshoot=np.zeros([image1.shape[0], image1.shape[1], image1.shape[2]], dtype='uint8')
    
    for i in range(j):        
        th=multi_otsu_1(a[i])  
        overshoot[i]=a[i]>th
        
        if removal_round==1: #larger obj removal size
       
            overshoot[i] = (morphology.remove_small_objects(label(overshoot[i]),min_size=30, connectivity=1)) #diff obj size to be removed for round 1 vs round 2
            overshoot[i]=cv2.morphologyEx(overshoot[i], cv2.MORPH_CLOSE, np.ones((5,5),np.uint8)) 
            th, overshoot[i] = cv2.threshold(overshoot[i], 0, 1, cv2.THRESH_BINARY)
        else:
            overshoot[i] = (morphology.remove_small_objects(label(overshoot[i]),min_size=20, connectivity=1)) #diff obj size to be removed for round 1 vs round 2
            th, overshoot[i] = cv2.threshold(overshoot[i], 0, 1, cv2.THRESH_BINARY)
        

    overshoot_removed=image1-overshoot #rough mask subtract the overshoot mask
    overshoot_removed[overshoot_removed==255] = 1 
  
    
    for i in range(j): 
        overshoot_removed[i]= (morphology.remove_small_objects(label(overshoot_removed[i]),min_size=30, connectivity=1)) #remove any bits remaining from subtraction
        th, overshoot_removed[i] = cv2.threshold(overshoot_removed[i].astype(np.uint8), 0, 1, cv2.THRESH_BINARY)

    return overshoot,overshoot_removed

def keep_overlaps(image,overlap_num):
    '''
    This function keeps parts of the image that is greater than overlap_num when thresholding
    '''
    image2=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    
    
    for i in range(j):
        if i==0:
            image2[i]=image[i]+image[i+1]
            image2[i][image2[i]==2]=3
        elif i==(j-1):
            image2[i]=image[i-1]+image[i]
            image2[i][image2[i]==2]=3
        else:
            image2[i]=image[i-1]+image[i]+image[i+1]
    

    ret, image2= cv2.threshold(image2,overlap_num,1,cv2.THRESH_BINARY) 
    for i in range(j):
        image2[i]= (morphology.remove_small_objects(label(image2[i]),min_size=40, connectivity=1)) 
    
    ret, image2= cv2.threshold(image2.astype(np.uint8),0,1,cv2.THRESH_BINARY)

    return image2

#### Snakes prep: Get Muscle Hull

In [59]:
def get_contours(im): 
    '''
    This function gets the contours of an image/mask
    '''
    contour_coords_L=[]

    mask_contours=np.zeros([Ivol8c_subcfat.shape[0], Ivol8c_subcfat.shape[1], Ivol8c_subcfat.shape[2]], dtype='uint8')
    zeros=np.zeros([Ivol8c_subcfat.shape[0], Ivol8c_subcfat.shape[1], Ivol8c_subcfat.shape[2]], dtype='uint8')
    for i in range (j):
        a, b =  cv2.findContours(im[i], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contour_coords_L.append([])
        contour_coords_L[i].append(a) 

        #draw contours from contour_coords_L onto zeros array 
        mask_contours[i] = cv2.drawContours(zeros[i], contour_coords_L[i][0],  -1, (1,0,0), 1)
        
    return mask_contours, contour_coords_L


def convex_hull(contours):
    '''
    This function gets rid of concavities in contour, used to follow the fascia as much as possible and initiate the snake
    '''
    hull_coords=[]
    hull_demonstration = np.zeros((Ivol8c_subcfat.shape[0], Ivol8c_subcfat.shape[1], Ivol8c_subcfat.shape[2], 3), np.uint8)
    hull = np.zeros((Ivol8c_subcfat.shape[0], Ivol8c_subcfat.shape[1], Ivol8c_subcfat.shape[2]), np.uint8)
    listt=[]
    contours_prehull=[]
    for i in range(j):
        hull_coords.append([])  
        listt.append([])
        contours_prehull.append([])

            
        for i2 in range(len(contours[i])):
            for i3 in range(len(contours[i][i2])):
                for i4 in range(len(contours[i][i2][i3])):
                    listt[i].append(contours[i][i2][i3][i4])
     
        contours_prehull[i]=np.array(listt[i])

        hull_coords[i].append(cv2.convexHull(contours_prehull[i], False))
    
        for i2 in range(len(contours[i])):
            color_contours = (0, 255, 0) # green - color for contours
            color_hull = (255, 0, 0) # red - color for convex hull
            cv2.drawContours(hull_demonstration[i], contours[i][0], i2, color_contours,
                             1, 8) 
            cv2.drawContours(hull_demonstration[i], hull_coords[i], i2, color_hull, 1, 8)

        for i2 in range(len(contours)): 
            color_hull = (255, 0, 0) # blue - color for convex hull
            cv2.drawContours(hull[i], hull_coords[i],  -1, (1,0,0), 1)
        
    return hull, hull_demonstration, hull_coords

def show_convexhull():
    '''
    This function displays the convex_hull
    '''
    fig, axs = plt.subplots(j, 3, figsize= (18, 120))
    for i in range(j):
        axs[i, 0].set_title(f"slice {i+1}", fontsize=18)
        axs[i, 0].imshow(mask_contours[i])
        axs[i, 1].imshow(hull_demonstration[i])
        axs[i, 2].imshow(hull[i])

        
def compatible_coordlist(contours):
    '''
    This function generate the initial coordinates to use for the snake
    '''
    initiator_coords=[]

    for i in range(j):
        initiator_coords.append([])
        for i2 in range(len(contours[i][0][0])): 
            initiator_coords[i].append(contours[i][0][0][i2][0])
        initiator_coords[i]=np.array(initiator_coords[i])
        initiator_coords[i]=initiator_coords[i].astype(float)
    return initiator_coords

#### Remove vessels

In [60]:
def remove_vessels(image): 
    '''
    This function removes the vessels from the ROI
    '''
    vessels_present=np.zeros([image.shape[0], image.shape[1], image.shape[2]], dtype='uint8')
    test=np.zeros([image.shape[0], image.shape[1], image.shape[2]], dtype='uint8')
    vessels_removed_mask=np.zeros([image.shape[0], image.shape[1], image.shape[2]], dtype='uint8')

    for i in range(j):
        otsu_th=multi_otsu_1(image[i])
        vessels_present[i] = image[i]>otsu_th
        
        vessels_removed_mask[i] = label(vessels_present[i]) 
  
        vessels_removed_mask[i]=cv2.morphologyEx(vessels_removed_mask[i], cv2.MORPH_CLOSE, np.ones((5,5),np.uint8)) 
    
        vessels_removed_mask[i] = (morphology.remove_small_holes(vessels_removed_mask[i],area_threshold=90, connectivity=1)).astype(int) 
        
        vessels_removed_mask[i]=cv2.normalize(src=vessels_removed_mask[i], dst=None, alpha=0.0, beta=1.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    return vessels_removed_mask, vessels_present
        

def get_vessels():
    '''
    This function checks the area to isolate circles which represent the vessels
    '''
    contour_coords_L=[]
    hiercvol=[]
    contour_listvol=[]
    boneroi=[]
    vessels_mask=[]

    r=np.zeros([Ivol8c_subcfat.shape[0], Ivol8c_subcfat.shape[1], Ivol8c_subcfat.shape[2]], dtype='uint8')

    for i in range(j):
        contour_coords_L.append([])
        hiercvol.append([])
        contour_listvol.append([])
        boneroi.append([])
        vessels_mask.append([])

    for i in range (j):
        a, b =  cv2.findContours(pre_vessels_mask[i], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contour_coords_L[i].append(a)
        hiercvol[i].append(b)

        for contour in contour_coords_L[i][0]:
            approx = cv2.approxPolyDP(contour,0.01*cv2.arcLength(contour,True),True)

            if (len(approx) > 3):  
                contour_listvol[i].append(contour)

        for i2 in range (len(contour_listvol[i])): 
            (x,y),radius = cv2.minEnclosingCircle(contour_listvol[i][i2]) #finds a circle of the minimum area enclosing a 2d point set
            center = (int(x),int(y))
            radius = int(radius)
            area = cv2.contourArea(contour_listvol[i][i2]) 
      
            #radius<50 to indicate small objects; area check relative to radius to check the circularity
            if (radius < 50) & (area> (0.1*(3.14*(radius**2))) and area<100): #appends to boneROI if the radius is less than the specified number (indicating blood vessels) 
                boneroi[i].append(contour_listvol[i][i2])#append to list if it satisfies the above conditions
                
        vessels_mask[i] = cv2.drawContours(r[i], boneroi[i],  -1, (1,0,0), 1)
    
    vessels_mask=floodfillall(vessels_mask)

    for i in range(j): 
        vessels_mask[i]=cv2.dilate(vessels_mask[i],np.ones((3,3),np.uint8),iterations = 1) 
   
    return vessels_mask

def get_whites():
    '''
    This function generates white mask to help guide the snake away from the thigh border and vessels (which are dark)
    '''
    whites=np.zeros([Ivol8c_subcfat.shape[0], Ivol8c_subcfat.shape[1], Ivol8c_subcfat.shape[2]], dtype='uint8')

    for i in range(j):
        #generate the white outside
        whites[i]=musc_fat_mask[i].copy() 
        whites[i][whites[i]==0] = 255
   
        whites[i][whites[i]==1] = 0
        whites[i][vessels_mask[i]==1]=255
    return whites 

#### Refine THIGH Muscle Mask: Snakes

In [61]:
def get_snakeim():
    '''
    This function applies a whites mask onto the filtered image to generate the image that the snake will be operating on
    '''
    snake_im=Ivol8c_c_s.copy()
    for i in range(j):
        snake_im[i][whites[i]==255] = 255 
        snake_im[i][subcfat_ring[i]==1] = 255  
    return snake_im

def bilat_fil_snake_im(image): 
    '''
    This function applies a bilateral filter to get rid of some noise
    '''
    bilateral_t=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    for i in range(j):
        bilateral_t[i] = cv2.bilateralFilter(image[i],20,35,35)
    return bilateral_t


#### Apply Snakes

In [62]:
from skimage.filters import gaussian


def primary_snake():
    '''
    This function finds the primary snake coordinates using active contour model
    '''
    snake1_coords=[]
    for i in range(j): 
        snake1_coords.append([])
        snake1_coords[i]=active_contour(gaussian(bilateral_t[i], 1,preserve_range=False), initiator_coords[i],alpha=0.01,gamma=20,w_edge=8,w_line=-0.5,max_num_iter=5,max_px_move=1)#test for fascia smoother

    return snake1_coords 


def snaketomask(coordinates):  
    '''
    This function uses the snake coordinates and makes a snake mask
    ''' 
    vol=[]

    for i in range(j):
        vol.append([])
        for i2 in range(len(coordinates[i])):
            vol[i].append([])
            vol[i][i2].append(coordinates[i][i2])

        vol[i]=np.rint(vol[i]).astype(int)

    drawsnake= np.zeros((Ivol8c_subcfat.shape[0], Ivol8c_subcfat.shape[1], Ivol8c_subcfat.shape[2]), np.uint8)
    
    #polylines function connects the dots to make it one smooth contour
    for i in range(j):
        cv2.polylines(drawsnake[i], [vol[i]], isClosed=True, color = (1, 0, 0) , thickness=1) 
    
    snake_mask=drawsnake.copy()

    h, w = subcfatvol[i].shape[:2]

    for i in range (j):
        mask = np.zeros((h+2, w+2), np.uint8)
        (x,y),radius = cv2.minEnclosingCircle(vol[i]) 
        cv2.floodFill(snake_mask[i], mask, (round(x),round(y)), 255)

    ret, snake_mask = cv2.threshold(snake_mask,0,1,cv2.THRESH_BINARY)
    return snake_mask


def keep_overlaps2(image,overlap_num): 
    '''
    This function keeps parts of the image that is greater than overlap_num when thresholding
    '''
    image2=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    for i in range(j):
        if i==0:
            image2[i]=image[i]+image[i+1]
            image2[i][image2[i]==2]=3
        elif i==(j-1):
            image2[i]=image[i-1]+image[i]
            image2[i][image2[i]==2]=3
        else:
            image2[i]=image[i-1]+image[i]+image[i+1]
    ret, image2= cv2.threshold(image2,overlap_num,1,cv2.THRESH_BINARY) 
    return image2

#### Bone Removal

In [ ]:
from PIL import Image
import numpy as np
def sharpen(image,factor):
    '''
    This function sharpens the input images by enhancing their sharpness using a specified factor
    '''
    sharpened=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    for i in range(j):

        cf=image[i].copy() 
        cf=Image.fromarray(cf, mode=None)
        enhancer = ImageEnhance.Sharpness(cf) #make border of cortical bone sharper

        sharpened[i]=enhancer.enhance(factor) 
    return sharpened


def get_boneprep(image):
    '''This function prepares bone images by applying various filters and processing steps to enhance bone structures and remove noise
    '''
    k=14
    bone_prep=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    bone_ots=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    
    subcfatmask=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    
    bone_prep_th=[]


    bone_prep=sharpen(image,5.0) 
    bone_prep=apply_CurvatureFilter(bone_prep, Ivol8c_roi) 

    subcfatmask=musc_fat_mask-musc_mask_final 
    for i in range(j):
    
        bone_prep[i] = cv2.bilateralFilter(bone_prep[i],15,20,45) 
            
            
        bone_prep_th.append([])
        bone_prep_th[i]=multi_otsu_1(bone_prep[i])
        bone_prep[i]=bone_prep[i]>bone_prep_th[i]

            
        bone_prep[i]=bone_prep[i]-subcfatmask[i]
        bone_prep[i][bone_prep[i]>1] = 0 
            

        bone_prep[i] = (morphology.remove_small_holes(bone_prep[i],area_threshold=100, connectivity=1)) 

        bone_prep[i] = label(bone_prep[i])

        bone_prep[i] = (morphology.remove_small_objects(bone_prep[i],min_size=30, connectivity=1))    
        
        
        th, bone_prep[i] = cv2.threshold(bone_prep[i], 0, 1, cv2.THRESH_BINARY)
   
            
    return bone_prep


In [64]:
def Z_connectivity_w_adj(image):
    '''
    This function checks the Z-connectivity among slices in a set of images with their adjacent slices and separates Z-connected parts from non-Z parts
    '''
    nonZ=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    Z=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    for i in range(j):
        if i==0: #if first slice only add with next slice
            combined_w_next=image[i+1]+image[i]
            ret, fatconnectedparts_next= cv2.threshold(combined_w_next,1,1,cv2.THRESH_BINARY)
            z_connection=fatconnectedparts_next

        elif i==(j-1): #if last slice only add with prev slice
            combined_w_prev=image[i-1]+image[i]
            ret, fatconnectedparts_prev= cv2.threshold(combined_w_prev,1,1,cv2.THRESH_BINARY) #must match both adj slices
            z_connection=fatconnectedparts_prev
        else: #add with both prev and next slice
            combined_w_prev=image[i-1]+image[i]
            combined_w_next=image[i]+image[i+1]
            ret, fatconnectedparts_prev= cv2.threshold(combined_w_prev,1,1,cv2.THRESH_BINARY)
            ret, fatconnectedparts_next= cv2.threshold(combined_w_next,1,1,cv2.THRESH_BINARY)

            z_connection=fatconnectedparts_prev+fatconnectedparts_next
            ret, z_connection= cv2.threshold(z_connection,1,1,cv2.THRESH_BINARY) 

        #find XY connections to Z connected parts
        coordinates= np.argwhere(z_connection==1)     
            
        im_ff=image[i].copy() 
        h, w = im_ff.shape[:2] 
        mask = np.zeros((h+2, w+2), np.uint8)

        for item in range(len(coordinates)): #floodfill in the coordinates of z-connectivity 
            cv2.floodFill(im_ff, mask, (coordinates[item][1],coordinates[item][0]), 2)

        #Remove small islands for Non-Z parts
        nonZ[i]=(im_ff==1)
   
        Z[i] = (im_ff==2)
        
    return Z,nonZ


def Z_connectivity_w_one(image, image2): 
    '''
    This function checks the Z-connectivity of slices in a set of images with one other image and separates Z-connected parts from non-Z parts
    '''
    nonZ=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    Z=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    
    for i in range(j):     
        combined=image[i]+image2
        ret, z_connection= cv2.threshold(combined,1,1,cv2.THRESH_BINARY)
        z_connection=z_connection
        
        #find XY connections to Z connected parts
        coordinates= np.argwhere(z_connection==1)    
            
        im_ff=image[i].copy()  
        h, w = im_ff.shape[:2] 
        mask = np.zeros((h+2, w+2), np.uint8)

        for item in range(len(coordinates)): #floodfill in the coordinates of z-connectivity 
            cv2.floodFill(im_ff, mask, (coordinates[item][1],coordinates[item][0]), 2)

        #Remove small islands for Non-Z parts
        nonZ[i]=(im_ff==1)
        Z[i] = (im_ff==2)
         
    return Z,nonZ

#### Find potential bone

In [65]:
def apply_mid_coord_one(image,y,x):  
    '''
    This function applies a flood fill operation starting from a specific coordinate (y, x) in the input image and returns the resulting filled area
    '''
    coord_applied=(image.copy()).astype(np.uint8)
    h, w = coord_applied.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)
    cv2.floodFill(coord_applied, mask, (y,x), 2)

    coord_applied=(coord_applied==2).astype(np.uint8)
    return coord_applied


def get_centroid(image):
    '''
    This function finds the centroid of the image
    '''
    coords= np.argwhere(image>0)  
    x = [p[0] for p in coords]
    y = [p[1] for p in coords]
    centroid = [round(sum(x) / len(coords)), round(sum(y) / len(coords))] 
    return centroid

def find_potentialbone(image):
    '''
    This function identifies potential bone regions in the input image by finding contours, filtering them based on area and circularity criteria, and applying flood fill operations
    '''
    contour_coords_L1=[]
    contour_coords_L2=[]
    bonemwvol=[]
 
    test1=[]
    test2=[]
  
    zeros=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8') 
    
    zeros1=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    zeros2=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8') 

    k=0
    for i in range(j):
        contour_coords_L1.append([])
        contour_coords_L2.append([])
        bonemwvol.append([])
        
        test1.append([])
        test2.append([])

    
        a, b =  cv2.findContours(image[i], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
        contour_coords_L1[i].append(a) #list of contours for all slices (each index = list for each slice)
        for contour in contour_coords_L1[i][0]: 
            test1[i] = cv2.drawContours(zeros1[i], contour_coords_L1[i][0],  -1, (1,0,0), 1)
            test1[i]=floodfill(test1[i]) 
            approx = cv2.arcLength(contour,True)
            area = cv2.contourArea(contour) 
            (x,y),radius = cv2.minEnclosingCircle(contour) 

            radius = int(radius)
            a=0.8*(radius**2) #keep objects that resemble 80% of a circular shape
            b=0.5*(radius*approx)//2 
        
            if (area>a) & (area>b) & (area>30):  
                contour_coords_L2[i].append(contour) #append the contours that meet the criteria
                test2[i] = cv2.drawContours(zeros2[i], contour_coords_L2[i],  -1, (1,0,0), 1) 
        bonemwvol=test2
    

    
    bonemwvol=floodfillall(bonemwvol) 
    
    
    
    return bonemwvol


In [66]:
def get_biggest_obj(image): #must be LABELLED image
    '''
    This function takes a labeled image as input and returns the largest object (with the maximum sum of pixel values) from the labeled image
    '''
    channels_isolated=[] 

    for i in range(np.amax(image)):
        channels_isolated.append((image==i+1).astype(np.uint8))

    sum_channel_L=[]
    for i in range(np.amax(image)):
        sum_channel_L.append([])
        sum_channel_L[i]=np.sum(channels_isolated[i])

    sum_channel_L2=sum_channel_L.copy()
    largest_sum = max(sum_channel_L2)  


    for i in range(np.amax(image)):
        if sum_channel_L[i]==largest_sum: 
            largest_obj=channels_isolated[i]

    return  largest_obj


def find_potentialbone2(image1): 
    '''
    This function identifies potential bone regions from a labeled image by finding the largest connected object in each slice of the labeled image. It then divides the slices into two halves and combines the largest objects from each half to obtain a final potential bone region
    '''
    z_labelled=label(image1>0).astype(np.uint8) 
        
    tib=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

    for i in range(j):
        tib[i]=get_biggest_obj(z_labelled[i])  #get biggest obj from boneprep_Z_labelled 
    
    #=======================================Divide slices in half===============================================#
    
    #Slice number
    j1=round(j/2)
    j2=j-j1

    #Tib
    tib1=tib[:j1,:,:]
    tib2=tib[j1:,:,:]
    
    #bone-prep - needed when apply coordinates for fib
    bone_prep1=bone_prep[:j1,:,:]
    bone_prep2=bone_prep[j1:,:,:] 

    #Ivol8c_roi - needed for overlaying to check later
    Ivol8c_roi1=Ivol8c_roi[:j1,:,:] 
    Ivol8c_roi2=Ivol8c_roi[j1:,:,:]

    
    #===========Add HALF of the images separately =========#


    #-----TIBIA----#
    tib1_added=np.zeros([Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8') 
    tib2_added=np.zeros([Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8') 
    
    for i in range(j1):
        tib1_added+=tib1[i]
    for i in range(j2):
        tib2_added+=tib2[i]
    
    tib1_added_keep=label(tib1_added==np.amax(tib1_added)) #keep pixels that overlap at least for ALL slices
    tib2_added_keep=label(tib2_added==np.amax(tib2_added))

    
    tib1_added_keep=get_biggest_obj(tib1_added_keep) #sometimes portion of tib is separated, results in two obj - take bigger obj
    tib2_added_keep=get_biggest_obj(tib2_added_keep)  
    
    
    #====================Get mid-point coords of tib and fib - use as seed for floodfill on bone_prep=======================================#
    tib1_mid_coord=get_centroid(tib1_added_keep)
    tib2_mid_coord=get_centroid(tib2_added_keep)

    

    bone_prep1_copy=bone_prep1.copy() 
    bone_prep2_copy=bone_prep2.copy() 
    

     #------------TIBIA-------------------------#

    #apply coord on image of interest    
    tibmarrow1=bone_prep1.copy()
    tibmarrow2=bone_prep2.copy()
    
    k=0
    
    for i in range(j1):
        tibmarrow1[i]=apply_mid_coord_one(tibmarrow1[i],tib1_mid_coord[1],tib1_mid_coord[0]) 
    for i in range(j2):
        tibmarrow2[i]=apply_mid_coord_one(tibmarrow2[i],tib2_mid_coord[1],tib2_mid_coord[0])

            
    #===================================ADD ISOLATED TIBS + FIBS======================================================================================


    tib_final=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    
    i2=0 #need new counter for second half of slices
    for i in range(j):
        if i<j1:#if part of first half of slices
            tib_final[i]=tibmarrow1[i]
        else: #if part of second half of slices
            tib_final[i]=tibmarrow2[i2]
            i2+=1

    return tib_final


#### Get cortical bone

In [67]:
def corticaloutline(image): 
    '''
    This function extracts the outline of the cortical bone from an input image, isolating the cortical bone region and removing noise
    '''
    cortical=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

    for i in range(j):
        th0=multi_otsu_0(image[i]) 
        cortical[i]=label(image[i]>th0) 
        ret, cortical[i] = cv2.threshold(cortical[i],0,1,cv2.THRESH_BINARY) 
        cortical[i] = (morphology.remove_small_holes(cortical[i],area_threshold=20, connectivity=1)) #remove noise, isolate cortical bone 
        
    return cortical


def merge_bones(cortical_mask,bonem_mask):   
    '''
    This function merges the cortical bone mask and the bone marrow mask, isolating the cortical bone region while removing any overlapping regions with the bone marrow
    '''
    k=8

    #=========== TIBIA REMOVAL==========#
    
    corticalt=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8') 

    bonem_mask_dilated=cv2.dilate(bonem_mask,np.ones((5,5),np.uint8),iterations = 1) #minimize obj left behind after subtraction
    
    t_bone=cortical_mask-bonem_mask_dilated
    t_bone[t_bone>1]=0
   
    
    for i in range(j):   
        corticalt[i] = (morphology.remove_small_holes(label(t_bone[i]),area_threshold=30, connectivity=0)) #get rid of lines 


        corticalt[i] = (morphology.remove_small_objects(label(corticalt[i]),min_size=1500, connectivity=0)) #remove white stuff in cortical bone 
  
        corticalt[i] = (morphology.remove_small_objects(label(corticalt[i]),min_size=1500, connectivity=0)) #remove white stuff in cortical bone 
     
        corticalt[i]=cv2.morphologyEx(corticalt[i], cv2.MORPH_CLOSE, np.ones((7,7),np.uint8)) #get rid of remaining holes  
     
    corticalt_final=floodfillall(corticalt)
    
    #if more than 1 obj - take the largest 
    labelled=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    for i in range(j):
        num_obj,labelled[i]=cv2.connectedComponents(corticalt_final[i])
        if num_obj>2: #if more than just 2 obj (background + 1 bone)
            corticalt_final[i]=get_biggest_obj(label(corticalt_final[i])) #must be labelled
       
            
        else:
            corticalt_final[i]=corticalt_final[i]
    
    corticalt_final=cv2.morphologyEx(corticalt_final, cv2.MORPH_CLOSE, np.ones((7,7),np.uint8)) 
    
 
    return corticalt_final

def get_roi(musc_mask,bone_mask,image):
    '''
    This function extracts the region of interest (ROI) from an input image based on the muscle mask and bone mask
    '''
    roi_mask=musc_mask-bone_mask 
    roi_mask[roi_mask>1] = 0 
    ret, roi_mask = cv2.threshold(roi_mask,0,1,cv2.THRESH_BINARY)
    
    roi=roi_mask*image
    return roi_mask, roi

<span style="font-size: 25px;"> ITSA 1st Round

In [68]:
def initial_th(image):
    '''
    This function calculates the initial threshold values for optimization using multi-otsu thresholding for each slice of the input image
    '''
    initial_th=[]

    for i in range(j):
        initial_th.append([])
        initial_th[i]=multi_otsu_1(image[i]) 

    return initial_th

def ITSA_no_Z(roivar, ots,size): 
    '''
    This function implements the ITSA (Iterative Threshold Selection Algorithm) without Z-connectivity check. 
    It iteratively adjusts the threshold value based on the mean signal intensity of the muscle and fat regions until convergence
    '''

    k=1
    ThPrev=0
    ThRev=ots  

    ThPrevlist=[ThPrev]  
    ThRevlist=[ThRev] 

    klist=[0] 
    matchlist=[] 
    while ThRev!=ThPrev: 
        
        ThPrev=ThRev
        
        prefatmask = label(roivar>ThRev) 
        prefatmask = np.uint8(prefatmask) 
        prefatmask = (morphology.remove_small_objects(prefatmask,min_size=size, connectivity=1))
        ret, fatmask = cv2.threshold(prefatmask,0,1,cv2.THRESH_BINARY)
  
        fatseg = fatmask*roivar 
        preMuscSegM=roivar-fatseg 
        MuscSegM=np.ma.masked_where(preMuscSegM == 0, preMuscSegM)
        FatSegM=np.ma.masked_where(fatseg==0,fatseg) 
        MuscSegI=np.mean(MuscSegM) 
        FatSegI=np.mean(FatSegM)
        ThRev=(1+((FatSegI-MuscSegI)/FatSegI))*MuscSegI 
        
        ThPrevlist.append(ThPrev) 
        ThRevlist.append(ThRev) 
        klist.append(k) 
        matchlist.append("No") 
        k+=1
        if k==50:
            break
       
        if ThRev==ThPrev:
            matchlist.append("Yes")
            table=QTable([klist,ThPrevlist,ThRevlist,matchlist],
            names=('Iteration','ThPrev','ThRev','ThRev=ThPrev?'))
  

            x=klist 
            y=ThRevlist

    return fatmask, fatseg, ThRev 
      
def print_th(th):
    '''
    This function prints all threshold values
    '''
    for i in range(j):
        print (f"Slice {i+1} th = {th[i]}") 
        
def apply_ITSA_no_Z(image,initial_th,size):
    '''
    This function applies the ITSA without Z-connectivity check to each slice of the input image using the initial threshold values calculated previously
    '''
    fatseg_mask=np.empty([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    fatseg=np.empty([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    
    optimized_th_L=[] #optimized thresholds
    
    for i in range(j): 
        fatseg_mask[i],fatseg[i],ThRev=ITSA_no_Z(image[i], initial_th[i],size)
        optimized_th_L.append(ThRev) 
        
    return optimized_th_L,fatseg_mask,fatseg


def subtract_fat1(fat1):
    '''
    This function subtracts the fat region obtained from the initial thresholding from the input image to obtain the region for further processing
    '''
    roi_for_S1R2=np.empty([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    roi_for_S1R2=roi-fat1 
    return roi_for_S1R2

def fatfinal_S1R1(roivar,th,size): 
    '''
    This function applies the final thresholding to obtain the fat region after subtracting the initial fat region from the input image
    '''
    fat1_mask=np.empty([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    fat1=np.empty([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    
    for i in range(j):
        fat1_mask[i] = label(roivar[i]>th[i]) 
        fat1_mask[i] = np.uint8(fat1_mask[i]) 
        fat1_mask[i] = (morphology.remove_small_objects(fat1_mask[i],min_size=size, connectivity=1)).astype(int)
        ret, fat1_mask[i] = cv2.threshold(fat1_mask[i],0,1,cv2.THRESH_BINARY) 

        fat1[i]= fat1_mask[i]*roivar[i]   

    return fat1_mask,fat1


<span style="font-size: 25px;"> ITSA 2nd Round 

In [69]:
def apply_th(th, image): 
    '''
    This function applies the threshold values obtained from optimization to the input image to generate binary masks for z-connectivity checking
    '''
    zcheck=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    for i in range(j): 
        zcheck[i]=(image[i]>th[i]).astype(int)
    return zcheck


def get_final_fatsegs_S1(fatfinal,th,fatfinal_mask,image):  
    '''
    This function retrieves the final fat segments from Set 1 without removing objects by comparing the fat region obtained after applying the optimized threshold values to the input image with the initial fat region
    '''
    fat2_mask=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    fat2=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    

    fat1_mask=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    fat1=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    

    for i in range(j):
        fat1_mask[i]=fatfinal[i]>th[i] 
        
        fat2_mask[i]= (fat1_mask[i]!=fatfinal_mask[i])

        
        fat1[i]=fat1_mask[i]*image[i]
        fat2[i]=fat2_mask[i]*image[i]
        
    return fat1_mask,fat1,fat2_mask,fat2



In [70]:
def ITSA_w_Z(i, roivar, initial_th, zcheck,size): 
    '''
    This function performs the Improved Threshold Selection Algorithm (ITSA) with Z-connectivity check for a specific slice i of the input image. 
    It iteratively adjusts the threshold value based on fat and muscle intensities and ensures Z-connectivity by incorporating neighboring slices
    '''
    k=1
    ThPrev_S2=0 
    ThRev_S2= initial_th[i] 

    x=0
    y=0
    
    ThPrevlist=[ThPrev_S2]  
    ThRevlist=[ThRev_S2] 

    klist=[0] 
    matchlist=[] 
    
    FatInt_L=["N/A"]
    MuscInt_L=["N/A"]
   
    
    while ThRev_S2!=ThPrev_S2:
        ThPrev_S2=ThRev_S2  
        prefatmask_S2 = (roivar[i]>ThRev_S2)
        prefatmask_S2 = np.uint8(prefatmask_S2)
        ret, fatmask_S2 = cv2.threshold(prefatmask_S2,0,1,cv2.THRESH_BINARY) 

        if i==0:
            fatcombined_next=zcheck[i+1]+fatmask_S2
            ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)
            z_connection=fatconnectedparts_next
   
        elif i==(j-1):
            fatcombined_prev=zcheck[i-1]+fatmask_S2
            ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
            z_connection=fatconnectedparts_prev
   
        else:
            fatcombined_prev=zcheck[i-1]+fatmask_S2
            fatcombined_next=fatmask_S2+zcheck[i+1]
            ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
            ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)

            z_connection=fatconnectedparts_prev+fatconnectedparts_next
            ret, z_connection= cv2.threshold(z_connection,0,1,cv2.THRESH_BINARY) 
        
        #find XY connections to Z connected parts
        coordinates= np.argwhere(z_connection ==1) 
        
        
        im_ff=fatmask_S2.copy()
        h, w = im_ff.shape[:2] 
        mask = np.zeros((h+2, w+2), np.uint8)
        for item in range(len(coordinates)):
            cv2.floodFill(im_ff, mask, (coordinates[item][1],coordinates[item][0]), 2)
        
        #Remove small islands for Non-Z parts
        nonZ =label(im_ff==1)
        nonZ_keep = (morphology.remove_small_objects(nonZ,min_size=size, connectivity=1))
        ret, nonZ_keep= cv2.threshold(np.uint8(nonZ_keep),0,1,cv2.THRESH_BINARY)

        Z = (im_ff==2)
  
        
        prefatseg1_S2=(Z+nonZ_keep) 
        fatseg1_S2=prefatseg1_S2*roivar[i]
        
        
        #Fat and Muscle Quantification

        preMuscSegP_S2=roivar[i]-fatseg1_S2
        
        MuscSegP_S2=np.ma.masked_where(preMuscSegP_S2 == 0, preMuscSegP_S2)
        FatSegP_S2=np.ma.masked_where(fatseg1_S2==0,fatseg1_S2) 
        MuscSegI_S2=np.mean(MuscSegP_S2)
        FatSegI_S2=np.mean(FatSegP_S2)
        
        
        if type(FatSegI_S2)== np.ma.core.MaskedConstant:
            ThRev_S2=ThPrev_S2
        else:
            ThRev_S2=(1+((FatSegI_S2-MuscSegI_S2)/FatSegI_S2))*MuscSegI_S2 
        
         
        ThPrevlist.append(ThPrev_S2) 
        ThRevlist.append(ThRev_S2)
        klist.append(k) 
        matchlist.append("No")
        
       
        FatInt_L.append(FatSegI_S2)
        
        MuscInt_L.append(MuscSegI_S2)

        
        k+=1
        if k==50:
            break
            
        if ThRev_S2==ThPrev_S2:
            matchlist.append("Yes")
            table=QTable([klist,ThPrevlist,ThRevlist,matchlist],
            names=('Iteration','ThPrev_S2','ThRev_S2','ThRev_S2=ThPrev_S2?'))

    thresholds_S2=ThRev_S2
    
    return prefatseg1_S2, fatseg1_S2, thresholds_S2


def apply_ITSA_w_Z(roivar, initial_th, zcheck,size):
    '''
    This function applies the ITSA with Z-connectivity check to all slices of the input image and returns the optimized threshold values and segmented fat regions
    '''
    fatseg_mask =np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    fatseg =np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8') 

    optimized_th_L =[] 
    for i in range(j):
        optimized_th_L .append([])
        fatseg_mask[i],fatseg[i], optimized_th_L[i]=ITSA_w_Z(i, roivar,initial_th,zcheck,size)  
    return optimized_th_L,fatseg_mask,fatseg   



#### Final Z-Connectivity Check

In [71]:
def final_zcheck(i,th,zcheck,size): 
    '''
    This function performs the final Z-connectivity check for slice i after applying the optimized threshold value th[i] and considering neighboring slices' fat masks represented by zcheck
    '''
    prefatmask_R3 = (roi[i]>th[i]) 

    fatmask_R3 = np.uint8(prefatmask_R3)
    
    if i==0:
        fatcombined_next=zcheck[i+1]+fatmask_R3
        ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)
        z_connection=fatconnectedparts_next      
    elif i==(j-1):
        fatcombined_prev=zcheck[i-1]+fatmask_R3  
        ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
        z_connection=fatconnectedparts_prev    
    else:
        fatcombined_prev=zcheck[i-1]+fatmask_R3
        fatcombined_next=zcheck[i+1]+fatmask_R3 

        ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
        ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)

        z_connection=fatconnectedparts_prev+fatconnectedparts_next
        ret, z_connection= cv2.threshold(z_connection,0,1,cv2.THRESH_BINARY)      

        
    coordinates= np.argwhere(z_connection ==1)      
    im_ff=fatmask_R3.copy()
    h, w = im_ff.shape[:2] 
    mask = np.zeros((h+2, w+2), np.uint8)
    for item in range(len(coordinates)):
        cv2.floodFill(im_ff, mask, (coordinates[item][1],coordinates[item][0]), 2)

    #Remove small islands for Non-Z parts
    nonZ =label(im_ff==1)
    nonZ_keep = (morphology.remove_small_objects(nonZ,min_size=size, connectivity=1))
    ret, nonZ_keep= cv2.threshold(np.uint8(nonZ_keep),0,1,cv2.THRESH_BINARY)

    Z = (im_ff==2)
   
    final_fat_mask=(Z+nonZ_keep) 
    final_fat=final_fat_mask*Ivol8c_roi[i] 

  
    return final_fat_mask, final_fat

def apply_final_zcheck(th,zcheck,size):
    '''
    This function applies the final Z-connectivity check to all slices using the optimized threshold values and Z-connectivity masks, resulting in the final segmented fat masks and fat regions
    '''
    fat_mask=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    fat=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    
    for i in range(j):
        fat_mask[i], fat[i]=final_zcheck(i,th,zcheck,size) 

    return  fat_mask, fat

In [72]:
def get_final_fatsegs(fatfinal,th,fatfinal_mask,image):
    '''
    This function calculates the final segmented fat masks (fatseg_mask_final_R1 and fatseg_mask_final_R2) and fat regions (fatseg_final_R1 and fatseg_final_R2) based on the optimized threshold values th applied to the final fat segmentation result fatfinal. 
    It considers the 3D connectivity check and the difference between the final fat mask and the initial fat mask
    '''
    fatseg_mask_final_R1=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    fatseg_final_R1=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    
    fatseg_final_R2=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    fatseg_mask_final_R2=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

    for i in range(j):
        fatseg_mask_final_R1[i]=fatfinal[i]>th[i] 
        fatseg_mask_final_R2[i]= (fatseg_mask_final_R1[i]!=fatfinal_mask[i])

        fatseg_final_R1[i]=fatseg_mask_final_R1[i]*image[i]
        fatseg_final_R2[i]=fatseg_mask_final_R2[i]*image[i]
    
    return fatseg_mask_final_R1,fatseg_final_R1,fatseg_mask_final_R2,fatseg_final_R2


def overlay_TWO_SEP(f1,f2,image):
    '''
    This function overlays the segmented fat regions f1 and f2 on the original image image for visualization purposes. 
    It plots each slice of the image along with the overlaid fat regions in separate subplots
    '''
    def overlayTWO_1(image, f1,f2,x):    
        overlay = np.ma.masked_where(f1 == 0, f1)
        overlay2= np.ma.masked_where(f2 == 0, f2)
        axs[x,1].imshow(image, cmap="bone")
        axs[x,1].imshow(overlay, cmap="autumn", vmin=0, vmax=1)
        axs[x,1].imshow(overlay2, cmap="bwr", vmin=0, vmax=1, alpha=1)
        axs[x, 1].set_title(f"slice {i+1}", fontsize=12)
    fig, axs = plt.subplots(j, 2, figsize=(12, 100))
    for i in range(j):
        axs[i,0].imshow(image[i], cmap='bone')
        overlayTWO_1(image[i],f1[i], f2[i],i) 

<span style="font-size: 25px;"> Quality check and tag export

In [2]:
def quality_buttons(ID,slic):
    '''
    This function is used to create buttons to identify which masks need manual adjustments
    '''
    ID = ID
    slic = slic

    def record_click(button_clicked):
        # Create a dictionary with the data to be recorded
        new_entry = {
            'ID': ID,
            'slice': slic,
            'Button Clicked': button_clicked
        }

        # Append the new entry to the DataFrame
        data.loc[len(data)] = new_entry
    # Create a button for each case
    button_a = widgets.Button(description="Good")
    button_b = widgets.Button(description="Overshoot")
    button_c = widgets.Button(description="Undershoot")
    button_d = widgets.Button(description="Incorrect Thigh Cut")

    # Attach the click event handlers for buttons A and B
    def button_a_click(_):
        global button_clicked
        button_clicked = "Good"
        record_click(button_clicked)

    def button_b_click(_):
        global button_clicked
        button_clicked = "Overshoot"
        record_click(button_clicked)

    def button_c_click(_):
        global button_clicked
        button_clicked = "Undershoot"
        record_click(button_clicked)

    def button_d_click(_):
        global button_clicked
        button_clicked = "Incorrect Thigh Cut"
        record_click(button_clicked)
        
    button_a.on_click(button_a_click)
    button_b.on_click(button_b_click)
    button_c.on_click(button_c_click)
    button_d.on_click(button_d_click)

    # Display the buttons
    display(button_a)
    display(button_b)
    display(button_c)
    display(button_d)

    # Initialize the button_clicked variable
    button_clicked = None



In [75]:
def quality_check(image1, image2, ID):
    '''
    This function overlays 2 images to check the quality of the masks created and identify those that need manual edits
    '''
    def check(im, to_overlay):
        overlay = np.ma.masked_where(to_overlay == 0, to_overlay)
        plt.imshow(im, cmap="bone")
        plt.imshow(overlay, cmap="hsv", vmin=0, vmax=1, alpha=0.5)
        plt.title(f"{ID}", fontsize=12)
        plt.show()

    j = len(image1)  

    for i in range(j):
        check(image1[i], image2[i])

        quality_buttons(ID,i+1)

In [72]:

import errno 
import shutil

def writetag():
    '''
    This function generates a tag file and saves it to a specified directory. 
    The tag file contains metadata and pixel data extracted from an input image array
    '''
    
    volsitk.GetOrigin()
    epais = 0.0 
    org_x = volsitk.GetOrigin()[0]
    org_y = volsitk.GetOrigin()[1]
    org_z = 0 
    org_x, org_y, org_z
    dimx = volsitk.GetSize()[0]
    dimy = volsitk.GetSize()[1]
    dimz = 1
    inc_x = inc_y = 1
    
    dir_h_x=1.0000
    dir_h_y=0.0000
    dir_h_z=0.0000
    dir_v_x=0.0000
    dir_v_y=1.0000
    dir_v_z=0.0000

    imfmscb = []
    for j in range(imfbonemsc.shape[0]):
        for i in range(imfbonemsc.shape[1]): 
            imfmscb.append(imfbonemsc[j,i])
    imfmscbytes = bytes(np.uint8(np.array(imfmscb)))

    serdesc = "Exp_"+stid
    
    drive = r'D:\OAI_Costa\Tags_need' #***change to directory you want images and tags saved to
    outtag = os.path.join(drive, "ToSegmentBW", stid, "OutputTags")
    
    try:
        os.makedirs(outtag)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise   
    file = open(outtag+"\\"+serdesc+"_org"+str(slc)+".tag", "wb")

    header = \
    "x:"+str(dimx)+" "+ \
    "y:"+str(dimy)+" "+ \
    "z:"+str(dimz)+" "+ \
    "type:BYTE \n" + \
    "org_x:"+str(org_x)+" "+ \
    "org_y:"+str(org_y)+" "+ \
    "org_z:"+str(org_z)+" \n"+ \
    "inc_x:"+str(inc_x)+" "+ \
    "inc_y:"+str(inc_y)+" "+ \
    "epais:"+str(epais)+" \n"+ \
    "dir_h_x:1.0000     dir_h_y:0.0000     dir_h_z:0.0000     \n"+ \
    "dir_v_x:0.0000     dir_v_y:1.0000     dir_v_z:0.0000     \n"+ \
    "\x0c"

    headerb = header.encode()
    file.write(headerb)
    file.write(imfmscbytes)
    file.close()


def writetofolder(output_filename):
    '''
    This function copies a target range of original DICOM images to a specified directory and renames them accordingly
    '''
    drive = r'D:\OAI_Costa\Tags_need'  #***change to directory you want images and tags saved to
    root2 = os.path.join(drive, "ToSegmentBW", stid)
    outpath2 = os.path.join(root2, "OutputTags")
    try:
        os.makedirs(outpath2)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
    
    origdcm_fn = output_filename
    shutsrc = os.path.join(origdcm_fn)
    serdesc = "Exp_"+stid
    shutdst = os.path.join(outpath2, serdesc+"_org"+str(slc)+'.dcm')
    shutil.copy(shutsrc, shutdst)
        
    return drive, root2



#### Import corrected tags

In [83]:
import unittest
import logging
import re
import vtk
import vtk.util.numpy_support as VN


def tagtoarray(path):
    '''
    This function reads tag files from a specified directory, extracts metadata, and converts the voxel data into a NumPy array
    '''
    i = 0
    count=0
    imagetag= []
    tagarray0 = []
    tagsln0 = []
    for file in os.listdir(path):
        if file.find('tag') > 0:
            filename=os.path.join(path, file)

            # get slice #
            slpos = file.find(".dcm.tag")
            if slpos > 1 and file[slpos-1].isdigit():
                # Collect all consecutive digits preceding ".dcm.tag"
                digit_str = ""
                index = slpos - 1

                while index >= 0 and file[index].isdigit():
                    digit_str = file[index] + digit_str
                    index -= 1

                slicenum = int(digit_str.replace("_", ""))
                tagsln0.append(slicenum)
            
            endOfHeaderChar = '\x0c'

            with open(filename) as f:
                text = f.read(1000)  

                if not endOfHeaderChar in text:
                # end of header character is not found, it is not a valid tag file
                   print("This is an invalid tag file.")
                else:
                    header = text.split(endOfHeaderChar)[0]
                    fields = re.split('[\n\t\r ]+', header)

                    dims = [0, 0, 0]
                    origin = [0.0, 0.0, 0.0]
                    spacing = [1.0, 1.0, 1.0]
                    axisX = [1.0, 0.0, 0.0]
                    axisY = [0.0, 1.0, 0.0]

                    for field in fields:
                      if not field:
                        continue
                      name, value = field.split(':')
                      if name == 'x':
                        dims[0] = int(value)
                      elif name == 'y':
                        dims[1] = int(value)
                      elif name == 'z':
                        dims[2] = int(value)
                      elif name == 'org_x':
                        origin[0] = float(value)
                      elif name == 'org_y':
                        origin[1] = float(value)
                      elif name == 'org_z':
                        origin[2] = float(value)
                      elif name == 'inc_x':
                        spacing[0] = float(value)
                      elif name == 'inc_y':
                        spacing[1] = float(value)
                      elif name == 'epais':
                        spacing[2] = float(value)
                      elif name == 'dir_h_x':
                        axisX[0] = float(value)
                      elif name == 'dir_h_y':
                        axisX[1] = float(value)
                      elif name == 'dir_h_z':
                        axisX[2] = float(value)
                      elif name == 'dir_v_x':
                        axisY[0] = float(value)
                      elif name == 'dir_v_y':
                        axisY[1] = float(value)
                      elif name == 'dir_v_z':
                        axisY[2] = float(value)
                      elif name == 'type':
                        # type is BYTE in tag files
                        if value != 'BYTE':
                          logging.warning('Voxel type in tag file is expected to be BYTE')
                      elif name == 'uid':
                        pass
                      elif name == 'chksum':
                        pass

                    headerInfo = {
                        'dims': dims,
                        'origin': origin,
                        'spacing': spacing,
                        'axisX': axisX,
                        'axisY': axisY,
                        'headerSize': len(header)+1
                        }


                filePath=filename

                scalarType = vtk.VTK_CHAR
                numberOfComponents = 1
                sliceSize = headerInfo['dims'][0] * headerInfo['dims'][1] * vtk.vtkDataArray.GetDataTypeSize(scalarType) * numberOfComponents
                headerSize = headerInfo['headerSize']
                totalFilesize = os.path.getsize(filePath)
                voxelDataSize = totalFilesize - headerSize
                maxNumberOfSlices = int(voxelDataSize/sliceSize)
                if headerInfo['dims'][2] > maxNumberOfSlices:
                    logging.error(f"Tag file is expected to contain {headerInfo['dims'][2]} slices but it has only {maxNumberOfSlices}")

                reader = vtk.vtkImageReader2()
                reader.SetFileName(filePath)
                reader.SetFileDimensionality(3)
                reader.SetDataExtent(0, headerInfo['dims'][0]-1, 0, headerInfo['dims'][1]-1, 0, headerInfo['dims'][2]-1)
      
                reader.SetDataScalarType(scalarType)
                reader.SetNumberOfScalarComponents(numberOfComponents)
                reader.SetHeaderSize(headerSize)
                reader.SetFileLowerLeft(True) # to match input from NRRD reader
                reader.Update()



                imageout = reader.GetOutput()
                rows, cols, sl = imageout.GetDimensions()

                sc = imageout.GetPointData().GetScalars()
                a = VN.vtk_to_numpy(sc)
                a = a.reshape(sl,cols,rows)

                imagetag.append([])

                imagetag[count] = a[0]


                
                count+=1

                
    return imagetag, tagsln0, axisX, axisY

<span style="font-size: 25px;"> Running ITSA with corrected tags

In [84]:
def getOG(filepath):
    '''
    This function reads a DICOM file from the specified filepath and returns the pixel data as a NumPy array
    '''
    dicom_data = pydicom.dcmread(filepath, force=True)
    # Extract the pixel data as a NumPy array
    pixel_array = dicom_data.pixel_array
    return pixel_array

<span style="font-size: 25px;"> Calculations

In [90]:
def calc2(F1_mask,F1_im,F2_mask,F2_im,BF_mask,BF_im,roi_mask,roi_im):
    '''
    This function calculates various metrics related to fat and muscle areas and volumes within specified regions of interest (ROIs). 
    It computes areas, volumes, percentages, and correction factors based on provided masks and intensity images, returning a comprehensive DataFrame containing the calculated metrics for each slice of the image data
    '''
    ROI_MuscFatAreaPix_L=[]
    ROI_MuscFatAreaMM_L=[]
    ROI_MuscFatVolMM_L=[]
    F1_AreaPix_slice=[]
    F1_AreaMM_slice=[]
    Musc1AreaPix_slice=[]
    Musc1AreaMM_slice=[]
    F1_Perc_slice=[]
    F1_VolMM_slice=[]
    Musc1VolMM_slice=[]
    F2_AreaPix_slice=[]
    F2_AreaMM_slice=[]
    Musc2AreaPix_slice=[]
    Musc2AreaMM_slice=[]
    F2Perc_slice=[]

    F2VolMM_slice=[]
    F2_MuscVolMMc=[]
    Musc2VolMM_slice=[]
    FatSegI_F1_slice=[]
    FatSegI_F2_slice=[]
    cfactor_slice=[]
    FatVolCombined_slice=[]
    F2_VolMM_c_slice=[]
    FatVolCombined_C_slice=[] 
    F2_Perc_c_slice=[]
    FatPercCombined_C_slice=[]
    
    BFAreaPix_slice=[]
    BF_AreaMM_slice=[]
    Musc3AreaPix_slice=[]
    Musc3AreaMM_slice=[]
    BF_Perc_NOTc_slice=[]
    BF_VolMM_slice=[]
    Musc3VolMM_slice=[]
    Musc3VolMM_slice_c=[]
  
    FatVolCombined_C_all=[]
    MuscVolCombined_all=[]
    
    FatPercAvgCombined_all=[] 

    for i in range(j):
        ROI_MuscFatAreaPix_L.append([])
        ROI_MuscFatAreaMM_L.append([])
        ROI_MuscFatVolMM_L.append([])
        F1_AreaPix_slice.append([])
        F1_AreaMM_slice.append([])
        Musc1AreaPix_slice.append([])
        Musc1AreaMM_slice.append([])
        F1_Perc_slice.append([])
        F1_VolMM_slice.append([])
        Musc1VolMM_slice.append([])
        F2_AreaPix_slice.append([])
        F2_AreaMM_slice.append([])
        Musc2AreaPix_slice.append([])
        Musc2AreaMM_slice.append([])
        F2Perc_slice.append([])
        FatPercCombined_C_slice.append([]) 

        F2VolMM_slice.append([])
        F2_MuscVolMMc.append([])
        Musc2VolMM_slice.append([])
        FatSegI_F1_slice.append([])
        FatSegI_F2_slice.append([])
        cfactor_slice.append([])
        FatVolCombined_slice.append([])
        F2_VolMM_c_slice.append([])
        FatVolCombined_C_slice.append([])
        F2_Perc_c_slice.append([])

        BFAreaPix_slice.append([])
        BF_AreaMM_slice.append([])
        Musc3AreaPix_slice.append([])
        Musc3AreaMM_slice.append([])
        BF_Perc_NOTc_slice.append([])
        BF_VolMM_slice.append([])
        Musc3VolMM_slice.append([])
        Musc3VolMM_slice_c.append([])
    
        FatVolCombined_C_all.append([]) 
        MuscVolCombined_all.append([])
        
        FatPercAvgCombined_all.append([])
    #MUSCLE + FAT AREA
        ROI_MuscFatAreaPix=np.sum(roi_mask[i]>0)   
        ROI_MuscFatAreaMM=ROI_MuscFatAreaPix*im_spacing[0]*im_spacing[1] 
        ROI_MuscFatVolMM=ROI_MuscFatAreaMM*im_spacing[2]
        
    #THRESHOLD 1 FAT
        musc_no_F1=(roi_mask[i]-F1_mask[i])*roi_im[i] 

    
        #FAT AREA
        F1_AreaPix=np.sum(F1_mask[i]>0) 
        F1_AreaMM=F1_AreaPix*im_spacing[0]*im_spacing[1]
        #MUSCLE AREA
        Musc1AreaPix=np.sum(musc_no_F1>0)
        Musc1AreaMM=Musc1AreaPix*im_spacing[0]*im_spacing[1]
        #FAT PERCENTAGE
        F1_Perc=F1_AreaPix*100/ROI_MuscFatAreaPix 
        #VOLUME OF EACH SLICE
        F1_VolMM=F1_AreaMM*im_spacing[2] #multiply by z -slice thickness
        Mus1cVolMM=Musc1AreaMM*im_spacing[2]

    #THRESHOLD 2 FAT
        #NOT including threshold 1 fat----------------------------------------------
        musc_no_F2=(roi_mask[i]-F2_mask[i])*roi_im[i]  

        
        #FAT AREA
        F2_AreaPix=np.sum(F2_mask[i] >0)
        F2_AreaMM=F2_AreaPix*im_spacing[0]*im_spacing[1]
        #MUSCLE AREA
        Musc2AreaPix=np.sum(musc_no_F2>0)
        Musc2AreaMM=Musc2AreaPix*im_spacing[0]*im_spacing[1]
        #FAT PERCENTAGE
        F2Perc=F2_AreaPix*100/ROI_MuscFatAreaPix 
        #VOLUME OF EACH SLICE
        F2VolMM=F2_AreaMM*im_spacing[2]
        Musc2VolMM=Musc2AreaMM*im_spacing[2]
        
    #FINAL FAT using Threshold 2 ----------------------------------------------------
        musc_no_BF=(roi_mask[i]-BF_mask[i])*roi_im[i]

        
        #FAT AREA -F2 here NOT CORRECTED 
        BFAreaPix=np.sum(BF_mask[i]>0) 
        BF_AreaMM=BFAreaPix*im_spacing[0]*im_spacing[1]
        
        #MUSCLE AREA
        Musc3AreaPix=np.sum(musc_no_BF>0)
        Musc3AreaMM=Musc3AreaPix*im_spacing[0]*im_spacing[1]
        
        #FAT PERCENTAGE -F2 here NOT CORRECTED 
        BF_Perc_NOTc=BFAreaPix*100/ROI_MuscFatAreaPix 
        
        #VOLUME OF EACH SLICE
        BF_VolMM=BF_AreaMM*im_spacing[2] 
        Musc3VolMM=Musc3AreaMM*im_spacing[2]
        
      
    #FAT CORRECTION - break
        if np.sum(F2_im[i] >0)==0: #if there is NO R2 refined fat just take th1 loop fat
            print(f"{ID} Slice {i+1} no R2 refined")
            #keep vars for final table
            F1_im_masked=np.ma.masked_where(F1_im[i]==0,F1_im[i]) 
            F2_im_masked=np.ma.masked_where(F2_im[i] ==0,F2_im[i]) 
            FatSegI_F1=np.mean(F1_im_masked) 
            FatSegI_F2=np.mean(F2_im_masked)
            cfactor=9999
            FatVolCombined=9999
            F2_VolMM_c=9999
            F2_Perc_c=9999
            
            #take F1 values only
            FatVolCombined_C=F1_VolMM 
            FatPercCombined_C=F1_Perc
            
            
        else:

            F1_im_masked=np.ma.masked_where(F1_im[i]==0,F1_im[i]) 
            F2_im_masked=np.ma.masked_where(F2_im[i] ==0,F2_im[i])
            FatSegI_F1=np.mean(F1_im_masked) 
            FatSegI_F2=np.mean(F2_im_masked)
            cfactor=(FatSegI_F2/FatSegI_F1) #fat correction factor
    
        
            FatVolCombined=F1_VolMM+F2VolMM #final volume not corrected
            F2_VolMM_c=F2VolMM*cfactor 
            FatVolCombined_C=F1_VolMM+F2_VolMM_c #final volume corrected
            
            F2_Perc_c=F2Perc*cfactor 
            FatPercCombined_C=F1_Perc+F2_Perc_c 
            
        ROI_MuscFatAreaPix_L[i]=ROI_MuscFatAreaPix
        ROI_MuscFatAreaMM_L[i]=ROI_MuscFatAreaMM
        ROI_MuscFatVolMM_L[i]=ROI_MuscFatVolMM 
        F1_AreaPix_slice[i]=F1_AreaPix
        F1_AreaMM_slice[i]=F1_AreaMM
        Musc1AreaPix_slice[i]= Musc1AreaPix
        Musc1AreaMM_slice[i]=Musc1AreaMM
        F1_Perc_slice[i]=F1_Perc 
        F1_VolMM_slice[i]=F1_VolMM 
        Musc1VolMM_slice[i]=Mus1cVolMM
        F2_AreaPix_slice[i]=F2_AreaPix
        F2_AreaMM_slice[i]=F2_AreaMM
        Musc2AreaPix_slice[i]=Musc2AreaPix
        Musc2AreaMM_slice[i]=Musc2AreaMM
        F2Perc_slice[i]=F2Perc
        FatPercCombined_C_slice[i]=FatPercCombined_C 
        
        F2VolMM_slice[i]=F2VolMM
        F2_MuscVolMMc[i] = F2VolMM * (1-cfactor) 
        Musc2VolMM_slice[i]=Musc2VolMM
        FatSegI_F1_slice[i]=FatSegI_F1
        FatSegI_F2_slice[i]=FatSegI_F2
        cfactor_slice[i]=cfactor
        FatVolCombined_slice[i]=FatVolCombined
        F2_VolMM_c_slice[i]=F2_VolMM_c
        FatVolCombined_C_slice[i]=FatVolCombined_C 
        F2_Perc_c_slice[i]=F2_Perc_c
        FatPercCombined_C_slice[i]=FatPercCombined_C

        BFAreaPix_slice[i]=BFAreaPix
        BF_AreaMM_slice[i]=BF_AreaMM
        Musc3AreaPix_slice[i]=Musc3AreaPix
        Musc3AreaMM_slice[i]=Musc3AreaMM
        BF_Perc_NOTc_slice[i]=BF_Perc_NOTc
        BF_VolMM_slice[i]=BF_VolMM
        Musc3VolMM_slice[i]=Musc3VolMM #not corrected  
        Musc3VolMM_slice_c[i]= Mus1cVolMM + (F2VolMM * (1-cfactor)) 
        
        FatVolCombined_C_all[i]=sum(FatVolCombined_C_slice) 
        MuscVolCombined_all[i]=sum(Musc3VolMM_slice)
        
        FatPercAvgCombined_all[i]=mean(FatPercCombined_C_slice)
        
        
    slice_num=[]
    slice_num=list(range(1,j+1))  
    
    slice_num2=(list(reversed(slice_num)))  

    OG_slice_num=slice_num
    

    
    
    #table for ALL data
    data = {'ID': ID,'Slice': slice_num,'ROI_MuscFatAreaPix':ROI_MuscFatAreaPix_L, 'ROI_MuscFatAreaMM':ROI_MuscFatAreaMM_L, 'ROI_MuscFatVolMM': ROI_MuscFatVolMM_L,
            'F1_AreaPix':F1_AreaPix_slice,'F1_AreaMM':F1_AreaMM_slice,'F1_MuscAreaPix':Musc1AreaPix_slice,'F1_MuscAreaMM':Musc1AreaMM_slice,
            'F1_Perc':F1_Perc_slice,'F1_VolMM':F1_VolMM_slice,'F1_MuscVolMM':Musc1VolMM_slice, 
            'F2_AreaPix':F2_AreaPix_slice,'F2_AreaMM':F2_AreaMM_slice,'Musc_noF2_AreaPix':Musc2AreaPix_slice,'musc_no_F2_AreaMM':Musc2AreaMM_slice,
            'F2_Perc_NOTc':F2Perc_slice,'F2_VolMM_NOTc':F2VolMM_slice,'F2_MuscVolMM':Musc2VolMM_slice,
            'F1_Intensity':FatSegI_F1_slice,'F2_Intensity':FatSegI_F2_slice,
            'F2_cfactor':cfactor_slice,'BF_FatVol_NOTc':FatVolCombined_slice,'F2_VolMM_c':F2_VolMM_c_slice,'F2_MuscVolMMc':F2_MuscVolMMc,
            'BF_FatVol_c':FatVolCombined_C_slice,
            'F2_Perc_c':F2_Perc_c_slice,'BF_Perc_c':FatPercCombined_C_slice,
          
            'BF_AreaPix_NOTc':BFAreaPix_slice,'BF_AreaMM^2_NOTc':BF_AreaMM_slice,'Musc_noBF_AreaPix_NOTc':Musc3AreaPix_slice,
            'musc_no_BF_AreaMM_NOTc':Musc3AreaMM_slice,'BF_Perc_NOTc':BF_Perc_NOTc_slice,
            'BF_VolMM_NOTc':BF_VolMM_slice,'Musc_noBF_VolMM_NOTc':Musc3VolMM_slice,
            'TOTAL_FatVol_c':FatVolCombined_C_all,'TOTAL_MuscVol':MuscVolCombined_all,'TOTAL_MuscVol_c':Musc3VolMM_slice_c}  

    data_table = pd.DataFrame(data, columns = ['ID','Slice','ROI_MuscFatAreaPix','ROI_MuscFatAreaMM', 'ROI_MuscFatVolMM',
                                                    'F1_AreaPix','F1_AreaMM','F1_MuscAreaPix','F1_MuscAreaMM','F1_Perc','F1_VolMM','F1_MuscVolMM', 
                                                    'F2_AreaPix','F2_AreaMM','Musc_noF2_AreaPix','musc_no_F2_AreaMM','F2_Perc_NOTc','F2_VolMM_NOTc',
                                                    'F1_Intensity','F2_Intensity','F2_cfactor','BF_FatVol_NOTc','F2_VolMM_c','F2_MuscVolMMc','BF_FatVol_c','F2_Perc_c','BF_Perc_c',
                                                    
                                                    'BF_AreaPix_NOTc','BF_AreaMM^2_NOTc','Musc_noBF_AreaPix_NOTc','musc_no_BF_AreaMM_NOTc','BF_Perc_NOTc','BF_VolMM_NOTc','Musc_noBF_VolMM_NOTc','TOTAL_FatVol_c','TOTAL_MuscVol_c'])


    print(f'{ID} total fatvol= {sum(FatVolCombined_C_slice)}\n')

    
    Total_FatVol=sum(FatVolCombined_C_slice) 
    Total_MuscVol=sum(Musc3VolMM_slice)
    TotalMuscFatVol=Total_FatVol+Total_MuscVol #corrected  
    Total_FatPerc=Total_FatVol/TotalMuscFatVol

    
    return data_table  



## Export results to excel

In [91]:
def append_df_to_sheet(to_append,excel_path,sheetname):
    '''
    This function reads a specific sheetname from an Excel file and concatenates the contents of this sheet with "to_append", ensuring that the index is ignored during concatenation to maintain a continuous index.
    '''
    df_excel = pd.read_excel((excel_path),sheet_name=sheetname) #read specific sheet name
    result = pd.concat([df_excel,to_append], ignore_index=True) #concatenate sheet contents with new df
    return result